In [1]:
!pip install nolds
!pip install einops
# !pip install reformer_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.2 MB/s eta 0:00:00


#### GLS

In [ ]:
import numpy as np
import pandas as pd
from nolds import lyap_e
from sklearn.preprocessing import MinMaxScaler


data = pd.read_csv(r'/kaggle/input/original-chaotic-data/Power.csv')
print(data)


# 计算指定的区域的最大Lyapunov指数
def calculate_special_Lyapunov(data,list_break_points):
    for i in range(1, len(list_break_points)-1):
        b_start = list_break_points[i-1]
        break_point = list_break_points[i]
        b_end = list_break_points[i+1]
        sum_L = 0

        M = data.shape[1]
        sum_g, sum_s = 0, 0
        for m in range(M):
            object_data = data.iloc[b_start:b_end,m]
            N_length = b_end - b_start
            break_point_psition = break_point - b_start

            left_data = object_data[:break_point_psition]
            right_data = object_data[break_point_psition:]

            global_LLE = lyap_e(object_data, emb_dim=10, matrix_dim=4)[0]

            LLE1 = lyap_e(left_data, emb_dim=10, matrix_dim=4)[0]
            LLE2 = lyap_e(right_data, emb_dim=10, matrix_dim=4)[0]
            combine_L = (LLE1 * break_point_psition + LLE2 * (N_length-break_point_psition)) / N_length
            L = global_LLE - combine_L
            sum_g += global_LLE
            sum_s += combine_L
            print('{}-{}-{}'.format(b_start,break_point,b_end),global_LLE,combine_L,L,'混乱程度降低了：{:.5}%'.format(L*100/global_LLE))
        print(sum_g,sum_s,'混乱程度降低了：{:.5}%'.format((sum_g-sum_s)*100/sum_g))
# calculate_special_Lyapunov(data,[0, 1500, 4000])


def AddNewBreakpoints(object_data, b_start, b_end, M, time_gap = 10, point_gap = 1000):
    '''
    :param object_data: 需要新添加的分割点的目标数据
    :param b_start: 开始的位置点
    :param b_end: 结束的位置点
    :param time_gap: 时间间隔
    :return: 位置点
    '''
    max_sum_L, find_b = 0, None
    object_N = b_end - b_start
    b_s, b_e = b_start + point_gap, b_end - point_gap       # 加上长度限制

    list_global_LLE = []
    for m in range(M):
        global_LLE = lyap_e(object_data.iloc[b_start:b_end,m], emb_dim=10, matrix_dim=4)[0]
        list_global_LLE.append(global_LLE)

    for i in range(b_s, b_e, time_gap):
        sum_L = 0
        for m in range(M):
            object_data_special = object_data.iloc[:,m].tolist()
            length = i - b_start      # 获得长度

            LLE1 = lyap_e(object_data_special[b_start:i], emb_dim=10, matrix_dim=4)[0]
            LLE2 = lyap_e(object_data_special[i:b_end], emb_dim=10, matrix_dim=4)[0]
            L = list_global_LLE[m] - (LLE1 * length + LLE2 * (object_N-length)) / object_N
            print(list_global_LLE[m],LLE1,LLE2,L)

            sum_L += L        # 对多变量的L求和，取大于0的最大值的对应的分割点
        if sum_L > max_sum_L:
            max_sum_L = sum_L
            find_b = i
        else:
            continue
    return find_b


# 在多变量中去切分
def Multi_Greedy_Lyapunov_Segment(data, point_gap=1000):
    N, M = data.shape
    break_points = []
    start, end = 0, N + 1
    current_breaks = [start, end]

    while current_breaks:
        new_breaks = []
        for i in range(len(current_breaks) - 1):
            print(current_breaks[i], current_breaks[i+1],data[current_breaks[i]:current_breaks[i+1]].shape,current_breaks)

            if current_breaks[i+1] - current_breaks[i] > point_gap*2:
                mid = AddNewBreakpoints(data, current_breaks[i], current_breaks[i+1],M, time_gap=50, point_gap=1000)
                if mid:
                    new_breaks.extend([current_breaks[i], mid, current_breaks[i+1]])
                    break_points.append(mid)
                else:
                    new_breaks.append(current_breaks[i])
            else:
                new_breaks.append(current_breaks[i])
                print('长度不够，没办法进行切分！')

        current_breaks = new_breaks
    print(break_points)
    return break_points
Multi_Greedy_Lyapunov_Segment(data.iloc[:,:])



# Chaotic-Net

## 消融实验

In [ ]:
# wo2_MoEE

import math
import torch
import torch.nn as nn
# from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal

import seaborn as sns
import matplotlib.pyplot as plt

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[
    torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """
    q = q.contiguous()
    k = k.contiguous()

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)


class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, out_channels=10, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()

        self.list_input_dim = list_input_dim
        self.list_conv = nn.ModuleList()
        for input_dim in list_input_dim:
            # 定义PwConv将不同的嵌入维数映射到相同的嵌入维数，并没有提取时间信息，某方面说提取到了特征之间的信息
            conv_layer = nn.Sequential(
                nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                          stride=emb_stride),
                nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=1)
                )
            self.list_conv.append(conv_layer)
        self.inst_norm = nn.InstanceNorm1d(out_channels)  # 实例化归一化层

    def forward(self, x):
        index = 0
        combine_features = torch.Tensor(0).to(device)
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            index += dim
            out = self.list_conv[i](input_x).to(device)

            combine_features = torch.cat((combine_features, out.unsqueeze(1)), dim=1)
        return combine_features

 
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, seq_len, dropout=0.1, gate_mlp=False):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((seq_len, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)
        self.gate_mlp = gate_mlp
        self.gate_linear = nn.Linear(d_in, d_hid)
        self.silu = nn.SiLU()

    def forward(self, x):
        residual = x

        if self.gate_mlp == False:
            x = self.w_2(F.relu(self.w_1(x)))

        else:
            x = self.w_2(F.relu(self.w_1(x) * (self.silu(self.gate_linear(x)))))

        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, seq_len, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads
        self.num_hiddens = num_hiddens

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, num_hiddens, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=num_hiddens // num_heads, seq_len=seq_len)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, self.num_hiddens // self.num_heads)
        k = xk.reshape(-1, time_len, self.num_hiddens // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d), dim=-1)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat)


class wo2_MoEE_Model(nn.Module):
    def __init__(self, M, time_len, list_input_dims, d=64, num_layers=1):
        super(wo2_MoEE_Model, self).__init__()
        self.M = M
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims

        # self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d, num_experts=num_experts, k=k)
        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d)

        time_lens= 30
        self.indenpendet_transformer = nn.ModuleList([MultiAttention(input_dim=d, num_hiddens=d, num_heads=8,seq_len=time_lens).to(device) for _ in range(M)])
        self.list_ln = nn.ModuleList([nn.LayerNorm(normalized_shape=(time_lens, d)) for _ in range(M)])
        self.indenpendet_FFN = nn.ModuleList([PositionwiseFeedForward(d, d, seq_len=time_lens,gate_mlp=True).to(device) for _ in range(M)])

        # self.dagcn = Mulit_DAGCN(num_time_steps=30, num_nodes=M, in_dims=d, out_dims=d, cheb_k=3, embed_dim=2)
        self.dagcn = Attention_DAGCN(num_time_steps=time_lens, num_nodes=M, embed_dim=5, in_dims=d,out_dims=d)
        self.ln2 = nn.LayerNorm(normalized_shape=(time_lens, d))

        self.weights = nn.Parameter(torch.ones(1) / 2)  # 可学习的权重参数

        self.FFN = PositionwiseFeedForward(d, d, seq_len=time_lens)
        self.FFN2 = PositionwiseFeedForward(d, d, seq_len=time_lens)

        self.linear_time = nn.Linear(time_lens, 1)
        self.linear_feat = nn.Linear(M * d, 1)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D

        for _ in range(self.num_layers):       
            list_independent_x = []
            for i in range(self.M):
                in_x = x[:, i, :, :]

                independent_x = self.indenpendet_transformer[i](in_x)
                independent_x = self.list_ln[i](in_x + independent_x)
                independent_x = self.indenpendet_FFN[i](independent_x) + in_x
                list_independent_x.append(independent_x.unsqueeze(1))
            
            independent_x = torch.cat(list_independent_x, dim=1)
            # independent_x = self.dagcn(independent_x.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)
            
            dagcn_x, scores, supports = self.dagcn(independent_x + x)
            x = self.FFN2(self.ln2(dagcn_x))+ x
            # x = self.FFN2(dagcn_x)+ x
            # x = independent_x + x

        out = self.linear_time(x.transpose(2, 3)).squeeze(-1)
        out = self.linear_feat(out.reshape(B, -1))
        return out, _, _, _, _

    def print_concat_weights(self):
        print('Independent Transformer and DAGCN concat weights:', self.dn_embeddings)


# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # B,L,D
#     model = wo2_MoEE_Model(time_len=32,M=3,list_input_dims=[5,5,5])
#     out, _, _, _, _ = model(x)
#     print(out.shape)


In [ ]:
# wo3_independent

import math
import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[
    torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)



class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, num_experts=8, out_channels=10, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()
        self.k = 2
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels
        self.list_input_dim = list_input_dim

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        for input_dim in list_input_dim:
            experts = nn.Sequential(
                nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                          stride=emb_stride),
                nn.Tanh(),
                nn.Conv1d(in_channels=out_channels, out_channels=out_channels * num_experts, kernel_size=1))

            # gating = nn.Linear(input_dim, num_experts)
            # noise = nn.Linear(input_dim,num_experts)

            w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

            self.list_m_experts.append(experts)
            self.list_m_gating.append(w_gate)
            self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, m, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[m]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[m]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_combine_features = []
        list_spatial_x = []
        list_gates = []
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            list_spatial_x.append(x[:, index, :].unsqueeze(1))
            index += dim

            B, d, L = input_x.shape
            # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
            gates, load = self.noisy_top_k_gating(input_x[:, :, -6:-1].reshape(B, dim * 5), i, self.training)
            list_gates.append(gates.unsqueeze(-1))

            # calculate importance loss
            importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
            loss = self.cv_squared(importance) + self.cv_squared(load)
            loss *= loss_coef
            out_loss = out_loss + loss

            out_raw = self.list_m_experts[i](input_x)  # B,D*E,l
            out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

            moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

            list_combine_features.append(moe_out.unsqueeze(1))
        combine_features = torch.cat(list_combine_features, dim=1)
        gates = torch.cat(list_gates, dim=-1)
        return combine_features, out_loss, gates


class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((30, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, input_dim, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=input_dim // num_heads, seq_len=30)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, input_dim // self.num_heads)
        k = xk.reshape(-1, time_len, input_dim // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d),dim=-1)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat)


class wo3_independent_Model(nn.Module):
    def __init__(self, M, time_len, list_input_dims, d=64, num_layers=1):
        super(wo3_independent_Model, self).__init__()
        self.M = M
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims

        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d)
        self.ln = nn.LayerNorm(normalized_shape=(30, d))

        self.mix_transformer = MultiAttention(input_dim=d, num_hiddens=d, num_heads=8)
        self.mix_FFN = PositionwiseFeedForward(d, d)

        # self.dagcn = Mulit_DAGCN(num_time_steps=30, num_nodes=M, in_dims=d, out_dims=d, cheb_k=3, embed_dim=2)
        self.dagcn = Attention_DAGCN(num_time_steps=30, num_nodes=M, embed_dim=3, in_dims=d,out_dims=d)

        self.weights = nn.Parameter(torch.ones(2) / 2)  # 可学习的权重参数

        self.FFN = PositionwiseFeedForward(d, d)

        self.linear_time = nn.Linear(30, 1)
        self.linear_feat = nn.Linear(M * d, 1)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x, loss, gates = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D

        for _ in range(self.num_layers):
            x = x.reshape(-1,x.shape[2],x.shape[3])
            mix_x = self.mix_transformer(x)
            mix_x = self.ln(x + mix_x)
            mix_x = self.mix_FFN(mix_x)
            x = mix_x.reshape(B,-1,x.shape[1],x.shape[2])

            mix_x,_,_ = self.dagcn(x)
            x = self.FFN(self.ln(mix_x + x))

        out = self.linear_time(x.transpose(2, 3)).squeeze(-1)
        out = self.linear_feat(out.reshape(B, -1))
        return out, loss, gates, _, _

    def print_concat_weights(self):
        print('Independent Transformer and DAGCN concat weights:', self.dn_embeddings)


# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # B,L,D
#     model = wo3_independent_Model(time_len=32,M=3,list_input_dims=[5,5,5])
#     out, _, gates = model(x)
#     print(out.shape)


In [ ]:
# wo4_Transformer

import math
import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.nn.utils import weight_norm
from torch.distributions.normal import Normal

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu



class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        """
        其实这就是一个裁剪的模块，裁剪多出来的padding
        """
        # print(x,x.shape)
        # print(x[:, :, :-self.chomp_size].contiguous(),x[:, :, :-self.chomp_size].contiguous().shape)
        # print('以进行裁剪')
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        """
        相当于一个Residual block

        :param n_inputs: int, 输入通道数
        :param n_outputs: int, 输出通道数
        :param kernel_size: int, 卷积核尺寸
        :param stride: int, 步长，一般为1
        :param dilation: int, 膨胀系数
        :param padding: int, 填充系数
        :param dropout: float, dropout比率
        """
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # 经过conv1，输出的size其实是(Batch, input_channel, seq_len + padding)
        self.chomp1 = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        """
        参数初始化
        :return:
        """
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        """
        :param x: size of (Batch, input_channel, seq_len)
        :return:
        """
        # print(self.conv1.weight.shape)
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs,seq_len, num_channels,pred_len, kernel_size=2, dropout=0.2):
        """
        TCN，目前paper给出的TCN结构很好的支持每个时刻为一个数的情况，即sequence结构，
        对于每个时刻为一个向量这种一维结构，勉强可以把向量拆成若干该时刻的输入通道，
        对于每个时刻为一个矩阵或更高维图像的情况，就不太好办。

        :param num_inputs: int， 输入通道数
        :param num_channels: list，每层的hidden_channel数，例如[25,25,25,25]表示有4个隐层，每层hidden_channel数为25，也就代表有25个卷积核在这次卷积中执行
        :param kernel_size: int, 卷积核尺寸，只需要定义卷积核长度
        :param dropout: float, drop_out比率
        """
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i  # 膨胀系数：1，2，4，8……
            in_channels = num_inputs if i == 0 else num_channels[i - 1]  # 确定每一层的输入通道数
            out_channels = num_channels[i]  # 确定每一层的输出通道数
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1,
                                     dilation=dilation_size,                        # 膨胀卷积
                                     padding=(kernel_size - 1) * dilation_size,     # 因果卷积
                                     dropout=dropout)]
        self.network = nn.Sequential(*layers)
        self.dense_pred = nn.Linear(seq_len,pred_len)
        self.dense_feature = nn.Linear(num_channels[-1],1)

    def forward(self, x):
        """
        输入x的结构不同于RNN，一般RNN的size为(Batch, seq_len, channels)或者(seq_len, Batch, channels)，
        这里把seq_len放在channels后面，把所有时间步的数据拼起来，当做Conv1d的输入尺寸，实现卷积跨时间步的操作，
        很巧妙的设计。

        为什么输入和输出数据结构是这样的
        :param x: size of (Batch, input_channel, seq_len)
        :return: size of (Batch, output_channel, seq_len)
        """
        x = x.permute(0,2,1)
        x = self.network(x)
        x = x.permute(0,2,1)
        return x



# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[
    torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)


class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, num_experts=8, out_channels=10, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()
        self.k = 2
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels
        self.list_input_dim = list_input_dim

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        for input_dim in list_input_dim:
            experts = nn.Sequential(
                nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                          stride=emb_stride),
                nn.Tanh(),
                nn.Conv1d(in_channels=out_channels, out_channels=out_channels * num_experts, kernel_size=1))

            # gating = nn.Linear(input_dim, num_experts)
            # noise = nn.Linear(input_dim,num_experts)

            w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

            self.list_m_experts.append(experts)
            self.list_m_gating.append(w_gate)
            self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, m, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[m]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[m]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_combine_features = []
        list_spatial_x = []
        list_gates = []
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            list_spatial_x.append(x[:, index, :].unsqueeze(1))
            index += dim

            B, d, L = input_x.shape
            # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
            gates, load = self.noisy_top_k_gating(input_x[:, :, -6:-1].reshape(B, dim * 5), i, self.training)
            list_gates.append(gates.unsqueeze(-1))

            # calculate importance loss
            importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
            loss = self.cv_squared(importance) + self.cv_squared(load)
            loss *= loss_coef
            out_loss = out_loss + loss

            out_raw = self.list_m_experts[i](input_x)  # B,D*E,l
            out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

            moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

            list_combine_features.append(moe_out.unsqueeze(1))
        combine_features = torch.cat(list_combine_features, dim=1)
        gates = torch.cat(list_gates, dim=-1)
        return combine_features, out_loss, gates


class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((30, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, input_dim, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=input_dim // num_heads, seq_len=30)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, input_dim // self.num_heads)
        k = xk.reshape(-1, time_len, input_dim // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d),dim=-1)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat)


class wo4_Transformer_Model(nn.Module):
    def __init__(self, M, time_len, list_input_dims,type_model='MLP', d=64, num_layers=1):
        super(wo4_Transformer_Model, self).__init__()
        self.M = M
        self.type_model = type_model
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims

        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d)
        self.ln = nn.LayerNorm(normalized_shape=(30, d))
        self.list_ln = nn.ModuleList([nn.LayerNorm(normalized_shape=(30, d)) for _ in range(M)])

        if type_model == 'MLP':
            self.indenpendet_layer = nn.ModuleList([PositionwiseFeedForward(d, d).to(device) for _ in range(M)])

        if type_model == 'LSTM':
            layer = nn.Sequential(nn.LSTM(d, d, batch_first=True))
            self.indenpendet_layer = nn.ModuleList([layer.to(device) for _ in range(M)])

        if type_model == 'TCN':
            layer = nn.Sequential(TemporalConvNet(num_inputs=d, num_channels=[d], kernel_size=2,seq_len=time_len,pred_len=1))
            self.indenpendet_layer = nn.ModuleList([layer.to(device) for _ in range(M)])

        # self.dagcn = Mulit_DAGCN(num_time_steps=30, num_nodes=M, in_dims=d, out_dims=d, cheb_k=3, embed_dim=2)
        self.dagcn = Attention_DAGCN(num_time_steps=30, num_nodes=M, embed_dim=3, in_dims=d,out_dims=d)

        self.weights = nn.Parameter(torch.ones(2) / 2)  # 可学习的权重参数

        self.FFN = PositionwiseFeedForward(d, d)

        self.linear_time = nn.Linear(30, 1)
        self.linear_feat = nn.Linear(M * d, 1)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x, loss, gates = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D

        for _ in range(self.num_layers):
            list_independent_x = []
            for i in range(self.M):
                if self.type_model == 'LSTM':
                    independent_x,_ = self.indenpendet_layer[i](x[:, i, :, :])
                else:
                    independent_x = self.indenpendet_layer[i](x[:, i, :, :])
                independent_x = self.list_ln[i](x[:, i, :, :] + independent_x)
                
                list_independent_x.append(independent_x.unsqueeze(1))
            independent_x = torch.cat(list_independent_x, dim=1)
            independent_x,_,_ = self.dagcn(independent_x)
            x = self.FFN(independent_x + x)+ x

        out = self.linear_time(x.transpose(2, 3)).squeeze(-1)
        out = self.linear_feat(out.reshape(B, -1))
        return out, loss, gates,_,_

    def print_concat_weights(self):
        print('Independent Transformer and DAGCN concat weights:', self.dn_embeddings)


# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # B,L,D
#     model = wo4_Transformer_Model(time_len=32,M=3,list_input_dims=[5,5,5],type_model='TCN')
#     out, _, gates = model(x)
#     print(out.shape)


In [ ]:
# wo5_DAGCN

import math
import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[
    torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)


class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, num_experts=8, out_channels=10, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()
        self.k = 2
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels
        self.list_input_dim = list_input_dim

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        for input_dim in list_input_dim:
            experts = nn.Sequential(
                nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                          stride=emb_stride),
                nn.Tanh(),
                nn.Conv1d(in_channels=out_channels, out_channels=out_channels * num_experts, kernel_size=1))

            # gating = nn.Linear(input_dim, num_experts)
            # noise = nn.Linear(input_dim,num_experts)

            w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

            self.list_m_experts.append(experts)
            self.list_m_gating.append(w_gate)
            self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, m, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[m]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[m]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_combine_features = []
        list_spatial_x = []
        list_gates = []
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            list_spatial_x.append(x[:, index, :].unsqueeze(1))
            index += dim

            B, d, L = input_x.shape
            # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
            gates, load = self.noisy_top_k_gating(input_x[:, :, -6:-1].reshape(B, dim * 5), i, self.training)
            list_gates.append(gates.unsqueeze(-1))

            # calculate importance loss
            importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
            loss = self.cv_squared(importance) + self.cv_squared(load)
            loss *= loss_coef
            out_loss = out_loss + loss

            out_raw = self.list_m_experts[i](input_x)  # B,D*E,l
            out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

            moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

            list_combine_features.append(moe_out.unsqueeze(1))
        combine_features = torch.cat(list_combine_features, dim=1)
        gates = torch.cat(list_gates, dim=-1)
        return combine_features, out_loss, gates


class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((30, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, input_dim, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=input_dim // num_heads, seq_len=30)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, input_dim // self.num_heads)
        k = xk.reshape(-1, time_len, input_dim // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d), dim=-1)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat)


class var_Attention1(nn.Module):
    def __init__(self, num_nodes, embed_dim, num_time_steps, in_dims, out_dims, num_layers=1):
        super(var_Attention1, self).__init__()

        self.w_q = nn.Conv2d(in_channels=num_nodes, out_channels=num_nodes, kernel_size=1, groups=num_nodes)
        self.w_k = nn.Conv2d(in_channels=num_nodes, out_channels=num_nodes, kernel_size=1, groups=num_nodes)

        self.w_v = nn.Conv2d(in_channels=num_nodes, out_channels=num_nodes, kernel_size=1)
        self.w_o = nn.Conv2d(in_channels=num_nodes, out_channels=num_nodes, kernel_size=1)

        self.num_time_steps = num_time_steps
        self.num_nodes = num_nodes
        self.embed_dim = embed_dim
        self.num_layers = num_layers

    def forward(self, x):
        B, M, L, D = x.shape

        for _ in range(self.num_layers):
            res_x = x
            unit = torch.eye(self.num_nodes).to(x.device)

            q = F.tanh(self.w_q(x)).reshape(B, M, -1)
            k = F.tanh(self.w_k(x)).reshape(B, M, -1)

            # q = self.w_q(x).reshape(B, M, -1)
            # k = self.w_k(x).reshape(B, M, -1)

            d = x.shape[-1]

            scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d), dim=-1).reshape(B, M, M) + unit  # B, M, M
            A = scores

            x = torch.einsum('bnm, bmld -> bnld', A, self.w_v(x))
            x = self.w_o(x) + res_x
        return x, _, _


class var_DAGCN(nn.Module):
    def __init__(self, num_nodes, embed_dim, num_time_steps, in_dims, out_dims, num_layers=1):
        super(var_DAGCN, self).__init__()
        self.num_time_steps = num_time_steps
        self.num_nodes = num_nodes
        self.embed_dim = embed_dim
        self.num_layers = num_layers

        # 动态空间节点嵌入向量
        self.dn_embeddings = nn.Parameter(0.5 * torch.ones(num_nodes,
                                                           embed_dim),
                                          requires_grad=True)  # [T, N, embed_dim]
        self.weights = nn.Parameter(torch.ones(1) / 2, requires_grad=True)  # 可学习的权重参数

        self.w_v = nn.Conv2d(in_channels=num_nodes, out_channels=num_nodes, kernel_size=1)
        self.w_o = nn.Conv2d(in_channels=num_nodes, out_channels=num_nodes, kernel_size=1)

    def forward(self, x):
        B, M, L, D = x.shape

        for _ in range(self.num_layers):
            res_x = x

            supports = F.softmax(F.relu(torch.einsum('ne, se -> ns', self.dn_embeddings, self.dn_embeddings)), dim=-1)
            unit = torch.eye(self.num_nodes).to(supports.device)
            supports = supports + unit

            A = supports
            x = torch.einsum('nm, bmld -> bnld', A, self.w_v(x))
            x = self.w_o(x) + res_x
        return x, _, _


class wo5_DCMC_Model(nn.Module):
    def __init__(self, M, time_len, list_input_dims, d=64, num_layers=1, model_name='DMCM'):
        super(wo5_DCMC_Model, self).__init__()
        self.M = M
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims
        self.model_name = model_name

        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d)
        self.ln = nn.LayerNorm(normalized_shape=(30, d))

        self.indenpendet_transformer = nn.ModuleList(
            [MultiAttention(input_dim=d, num_hiddens=d, num_heads=8).to(device) for _ in range(M)])
        self.indenpendet_FFN = nn.ModuleList([PositionwiseFeedForward(d, d).to(device) for _ in range(M)])

        if model_name == 'AGCN':
            self.layer = var_DAGCN(num_time_steps=30, num_nodes=M, embed_dim=3, in_dims=d,out_dims=d)
        if model_name == 'Attention':
            self.layer = var_Attention1(num_time_steps=30, num_nodes=M, embed_dim=3, in_dims=d,out_dims=d)

        self.FFN = PositionwiseFeedForward(d, d)

        self.weights = nn.Parameter(torch.ones(2) / 2)  # 可学习的权重参数

        self.linear_time = nn.Linear(30, 1)
        self.linear_feat = nn.Linear(M * d, 1)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x, loss, gates = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D

        for _ in range(self.num_layers):
            list_independent_x = []
            for i in range(self.M):
                # print(x.shape)
                independent_x = self.indenpendet_transformer[i](x[:, i, :, :])
                independent_x = self.ln(x[:, i, :, :] + independent_x)
                independent_x = self.indenpendet_FFN[i](independent_x)
                list_independent_x.append(independent_x.unsqueeze(1))
            independent_x = torch.cat(list_independent_x, dim=1)

            if self.model_name == 'AGCN':
                dagcn_x,_,_ = self.layer(independent_x + x)
            if self.model_name == 'Attention':
                dagcn_x,_,_ = self.layer(independent_x + x)
            
            # if self.model_name == 'DMCM':
            else:
                dagcn_x = independent_x + x

            x = self.FFN(self.ln(dagcn_x) + x) + x

        out = self.linear_time(x.transpose(2, 3)).squeeze(-1)
        out = self.linear_feat(out.reshape(B, -1))
        return out, loss, gates, _, _

    def print_concat_weights(self):
        print('Independent Transformer and DAGCN concat weights:', self.dn_embeddings)


# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # B,L,D
#     model = wo5_DCMC_Model(time_len=32,M=3,list_input_dims=[5,5,5],type_name='Attention')
#     out, _, gates,_ ,_ = model(x)
#     print(out.shape)


In [ ]:
# wo6_Expert_Linear

import math
import torch
import torch.nn as nn
# from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal

import seaborn as sns
import matplotlib.pyplot as plt

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[
    torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """
    q = q.contiguous()
    k = k.contiguous()

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)


class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, num_experts=4, out_channels=10,k=4, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()
        self.k = k
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels
        self.list_input_dim = list_input_dim

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        for input_dim in list_input_dim:
            experts = nn.Sequential(
                # nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                #           stride=emb_stride),
                # nn.Tanh(),
                # nn.Conv1d(in_channels=out_channels, out_channels=out_channels , kernel_size=1)
                nn.Linear(input_dim,out_channels * num_experts)
            )
            w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

            self.list_m_experts.append(experts)
            self.list_m_gating.append(w_gate)
            self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, m, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[m]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[m]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_combine_features = []
        list_spatial_x = []
        list_gates = []
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            list_spatial_x.append(x[:, index, :].unsqueeze(1))
            index += dim

            B, d, L = input_x.shape
            # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
            gates, load = self.noisy_top_k_gating(input_x[:, :, -6:-1].reshape(B, dim * 5), i, self.training)
            list_gates.append(gates.unsqueeze(-1))

            # calculate importance loss
            importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
            loss = self.cv_squared(importance) + self.cv_squared(load)
            loss *= loss_coef
            out_loss = out_loss + loss

            out_raw = self.list_m_experts[i](input_x.permute(0,2,1)).permute(0,2,1)  # B,D*E,l
            out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

            moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

            list_combine_features.append(moe_out.unsqueeze(1))
        combine_features = torch.cat(list_combine_features, dim=1)
        gates = torch.cat(list_gates, dim=-1)
        return combine_features, out_loss, gates


class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, seq_len, dropout=0.1, gate_mlp=False):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((seq_len, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)
        self.gate_mlp = gate_mlp
        self.gate_linear = nn.Linear(d_in, d_hid)
        self.silu = nn.SiLU()

    def forward(self, x):
        residual = x

        if self.gate_mlp == False:
            x = self.w_2(F.relu(self.w_1(x)))

        else:
            x = self.w_2(F.relu(self.w_1(x) * (self.silu(self.gate_linear(x)))))

        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, seq_len, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads
        self.num_hiddens = num_hiddens

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, num_hiddens, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=num_hiddens // num_heads, seq_len=seq_len)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, self.num_hiddens // self.num_heads)
        k = xk.reshape(-1, time_len, self.num_hiddens // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d), dim=-1)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat)


class wo6_Expert_Linear(nn.Module):
    def __init__(self, M, time_len, list_input_dims, d=64, num_layers=1,num_experts=2, k=2):
        super(wo6_Expert_Linear, self).__init__()
        self.M = M
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims

        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d, num_experts=num_experts, k=k)

        time_lens= 32
        self.indenpendet_transformer = nn.ModuleList([MultiAttention(input_dim=d, num_hiddens=d, num_heads=8,seq_len=time_lens).to(device) for _ in range(M)])
        self.list_ln = nn.ModuleList([nn.LayerNorm(normalized_shape=(time_lens, d)) for _ in range(M)])
        self.indenpendet_FFN = nn.ModuleList([PositionwiseFeedForward(d, d, seq_len=time_lens,gate_mlp=True).to(device) for _ in range(M)])

        # self.dagcn = Mulit_DAGCN(num_time_steps=30, num_nodes=M, in_dims=d, out_dims=d, cheb_k=3, embed_dim=2)
        self.dagcn = Attention_DAGCN(num_time_steps=time_lens, num_nodes=M, embed_dim=5, in_dims=d,out_dims=d)
        self.ln2 = nn.LayerNorm(normalized_shape=(time_lens, d))

        self.weights = nn.Parameter(torch.ones(1) / 2)  # 可学习的权重参数

        self.FFN = PositionwiseFeedForward(d, d, seq_len=time_lens)
        self.FFN2 = PositionwiseFeedForward(d, d, seq_len=time_lens)

        self.linear_time = nn.Linear(time_lens, 1)
        self.linear_feat = nn.Linear(M * d, 1)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x, loss, gates = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D

        for _ in range(self.num_layers):       
            list_independent_x = []
            for i in range(self.M):
                in_x = x[:, i, :, :]

                independent_x = self.indenpendet_transformer[i](in_x)
                # independent_x = self.list_ln[i](in_x + independent_x)
                # independent_x = self.indenpendet_FFN[i](independent_x) + independent_x
                independent_x = self.indenpendet_FFN[i](independent_x) + in_x
                list_independent_x.append(independent_x.unsqueeze(1))
            
            independent_x = torch.cat(list_independent_x, dim=1)
            # independent_x = self.dagcn(independent_x.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)
            
            dagcn_x, scores, supports = self.dagcn(independent_x + x)
            x = self.FFN2(self.ln2(dagcn_x))+ x
            # x = self.FFN2(dagcn_x)+ x
            # x = independent_x + x

        out = self.linear_time(x.transpose(2, 3)).squeeze(-1)
        out = self.linear_feat(out.reshape(B, -1))
        return out, loss, gates, scores, supports

    def print_concat_weights(self):
        print('Independent Transformer and DAGCN concat weights:', self.dn_embeddings)


In [ ]:
# wo7_Expert_MLP

import math
import torch
import torch.nn as nn
# from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal

import seaborn as sns
import matplotlib.pyplot as plt

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[
    torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """
    q = q.contiguous()
    k = k.contiguous()

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)


class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, num_experts=4, out_channels=10,k=4, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()
        self.k = k
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels
        self.list_input_dim = list_input_dim

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        for input_dim in list_input_dim:
            experts = nn.Sequential(
                nn.Linear(input_dim,out_channels),
                nn.Tanh(),
                nn.Linear(out_channels,out_channels * num_experts)

            )
            w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

            self.list_m_experts.append(experts)
            self.list_m_gating.append(w_gate)
            self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, m, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[m]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[m]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_combine_features = []
        list_spatial_x = []
        list_gates = []
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            list_spatial_x.append(x[:, index, :].unsqueeze(1))
            index += dim

            B, d, L = input_x.shape
            # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
            gates, load = self.noisy_top_k_gating(input_x[:, :, -6:-1].reshape(B, dim * 5), i, self.training)
            list_gates.append(gates.unsqueeze(-1))

            # calculate importance loss
            importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
            loss = self.cv_squared(importance) + self.cv_squared(load)
            loss *= loss_coef
            out_loss = out_loss + loss

            out_raw = self.list_m_experts[i](input_x.permute(0,2,1)).permute(0,2,1)  # B,D*E,l
            out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

            moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

            list_combine_features.append(moe_out.unsqueeze(1))
        combine_features = torch.cat(list_combine_features, dim=1)
        gates = torch.cat(list_gates, dim=-1)
        return combine_features, out_loss, gates


class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, seq_len, dropout=0.1, gate_mlp=False):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((seq_len, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)
        self.gate_mlp = gate_mlp
        self.gate_linear = nn.Linear(d_in, d_hid)
        self.silu = nn.SiLU()

    def forward(self, x):
        residual = x

        if self.gate_mlp == False:
            x = self.w_2(F.relu(self.w_1(x)))

        else:
            x = self.w_2(F.relu(self.w_1(x) * (self.silu(self.gate_linear(x)))))

        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, seq_len, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads
        self.num_hiddens = num_hiddens

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, num_hiddens, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=num_hiddens // num_heads, seq_len=seq_len)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, self.num_hiddens // self.num_heads)
        k = xk.reshape(-1, time_len, self.num_hiddens // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        scores = F.softmax(torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d), dim=-1)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat)


class wo7_Expert_MLP(nn.Module):
    def __init__(self, M, time_len, list_input_dims, d=64, num_layers=1,num_experts=2, k=2):
        super(wo7_Expert_MLP, self).__init__()
        self.M = M
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims

        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d, num_experts=num_experts, k=k)

        time_lens= 32
        self.indenpendet_transformer = nn.ModuleList([MultiAttention(input_dim=d, num_hiddens=d, num_heads=8,seq_len=time_lens).to(device) for _ in range(M)])
        self.list_ln = nn.ModuleList([nn.LayerNorm(normalized_shape=(time_lens, d)) for _ in range(M)])
        self.indenpendet_FFN = nn.ModuleList([PositionwiseFeedForward(d, d, seq_len=time_lens,gate_mlp=True).to(device) for _ in range(M)])

        # self.dagcn = Mulit_DAGCN(num_time_steps=30, num_nodes=M, in_dims=d, out_dims=d, cheb_k=3, embed_dim=2)
        self.dagcn = Attention_DAGCN(num_time_steps=time_lens, num_nodes=M, embed_dim=5, in_dims=d,out_dims=d)
        self.ln2 = nn.LayerNorm(normalized_shape=(time_lens, d))

        self.weights = nn.Parameter(torch.ones(1) / 2)  # 可学习的权重参数

        self.FFN = PositionwiseFeedForward(d, d, seq_len=time_lens)
        self.FFN2 = PositionwiseFeedForward(d, d, seq_len=time_lens)

        self.linear_time = nn.Linear(time_lens, 1)
        self.linear_feat = nn.Linear(M * d, 1)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x, loss, gates = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D

        for _ in range(self.num_layers):       
            list_independent_x = []
            for i in range(self.M):
                in_x = x[:, i, :, :]

                independent_x = self.indenpendet_transformer[i](in_x)
                # independent_x = self.list_ln[i](in_x + independent_x)
                # independent_x = self.indenpendet_FFN[i](independent_x) + independent_x
                independent_x = self.indenpendet_FFN[i](independent_x) + in_x
                list_independent_x.append(independent_x.unsqueeze(1))
            
            independent_x = torch.cat(list_independent_x, dim=1)
            # independent_x = self.dagcn(independent_x.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)
            
            dagcn_x, scores, supports = self.dagcn(independent_x + x)
            x = self.FFN2(self.ln2(dagcn_x))+ x
            # x = self.FFN2(dagcn_x)+ x
            # x = independent_x + x

        out = self.linear_time(x.transpose(2, 3)).squeeze(-1)
        out = self.linear_feat(out.reshape(B, -1))
        return out, loss, gates, scores, supports

    def print_concat_weights(self):
        print('Independent Transformer and DAGCN concat weights:', self.dn_embeddings)


## model

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


class DAGCN(nn.Module):
    def __init__(self, num_time_steps, num_nodes, in_dims, out_dims, cheb_k, embed_dim):
        super(DAGCN, self).__init__()
        self.num_time_steps = num_time_steps
        self.num_nodes = num_nodes
        self.cheb_k = cheb_k
        self.in_dims = in_dims
        self.out_dims = out_dims
        self.embed_dim = embed_dim

        # 动态空间节点嵌入向量
        self.dn_embeddings = nn.Parameter(0.01 * torch.zeros(num_time_steps,
                                                             num_nodes,
                                                             embed_dim),
                                          requires_grad=True)  # [T, N, embed_dim]
        # Theta = E*W--->(tnd,d)
        self.weights_pool = nn.Parameter(torch.randn(embed_dim,
                                                     cheb_k,
                                                     in_dims,
                                                     out_dims))
        self.bias_pool = nn.Parameter(torch.randn(embed_dim, out_dims))

    def forward(self, x):  # x-->[B,T,N,C]
        supports = F.softmax(F.relu(torch.einsum('tne, tse->tns', self.dn_embeddings, self.dn_embeddings)), dim=-1)

        unit = torch.stack([torch.eye(self.num_nodes).to(supports.device) for _ in range(self.num_time_steps)])
        support_set = [unit, supports]

        # default cheb_k = 3
        for k in range(2, self.cheb_k):
            support_set.append(torch.einsum('tnn, tns->tns', 2 * supports, support_set[-1]) - support_set[-2])
        supports = torch.stack(support_set, dim=1)  # [T, cheb_k, N, N]
        # print(supports.shape)

        # theta
        theta = torch.einsum('tnd, dkio->tnkio', self.dn_embeddings, self.weights_pool)  # T, N, cheb_k, dim_in, dim_out
        bias = torch.einsum('tnd, do->tno', self.dn_embeddings, self.bias_pool)  # T, N, dim_out
        x_g = torch.einsum('tknm, btmc->btknc', supports, x)
        x_gconv = torch.einsum('btkni, tnkio->btno', x_g, theta) + bias
        return x_gconv  # [B, T, N, dim_out]


# 多层GCN的层数（一般2到3层最好）
class Mulit_DAGCN(nn.Module):
    def __init__(self, num_time_steps, num_nodes, in_dims, out_dims, cheb_k, embed_dim, num_layers=1):
        super(Mulit_DAGCN, self).__init__()
        assert num_layers >= 1, 'At least one DCRNN layer in the Encoder.'
        self.node_num = num_nodes
        self.input_dim = in_dims
        self.num_layers = num_layers
        self.dcrnn_cells = nn.ModuleList()  # 模型列表
        self.dcrnn_cells.append(DAGCN(num_time_steps, num_nodes, in_dims, out_dims, cheb_k, embed_dim))
        for _ in range(1, num_layers):  # 定义多层
            self.dcrnn_cells.append(DAGCN(num_time_steps, num_nodes, in_dims, out_dims, cheb_k, embed_dim))

    def forward(self, x):
        state = None
        for i in range(self.num_layers):
            x = self.dcrnn_cells[i](x)
        return x  # torch.Size([71, 100, 1])


class Attention_DAGCN(nn.Module):
    def __init__(self, num_nodes, embed_dim, num_time_steps, num_layers=2, is_Dimension_Reduction=False, out_var_dims = 3):
        super(Attention_DAGCN, self).__init__()

        if is_Dimension_Reduction == False:
            out_var_dims = num_nodes
            # self.w_q = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=3, groups=num_nodes)
            # self.w_k = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=3, groups=num_nodes)
            
            self.w_q = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=1)
            self.w_k = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=1)

        else:
            out_var_dims = out_var_dims
            self.w_q = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=3)
            self.w_k = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=3)

        self.w_v = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=1)
        self.w_o = nn.Conv2d(in_channels=out_var_dims, out_channels=out_var_dims, kernel_size=1)
        self.w_res = nn.Conv2d(in_channels=num_nodes, out_channels=out_var_dims, kernel_size=1)

        self.num_time_steps = num_time_steps
        self.out_var_dims = out_var_dims
        self.embed_dim = embed_dim
        self.num_layers = num_layers

        # 动态空间节点嵌入向量
        self.dn_embeddings = nn.Parameter(torch.randn(out_var_dims,
                                                           embed_dim),
                                          requires_grad=True)  # [T, N, embed_dim]
        self.weights = nn.Parameter(torch.randn(1) / 2, requires_grad=True)  # 可学习的权重参数

    def forward(self, x):
        B, M, L, D = x.shape
        scores, supports = None, None

        res_x = x
        for _ in range(self.num_layers):
            supports = F.softmax(F.relu(torch.einsum('ne, se -> ns', self.dn_embeddings, self.dn_embeddings)), dim=-1)
            unit = torch.eye(self.out_var_dims).to(supports.device)
            supports = supports + unit

            q = F.tanh(self.w_q(x)).reshape(B, self.out_var_dims, -1)
            k = F.tanh(self.w_k(x)).reshape(B, self.out_var_dims, -1)

            # q = self.w_q(x).reshape(B, self.out_var_dims, -1)
            # k = self.w_k(x).reshape(B, self.out_var_dims, -1)

            d = x.shape[-1]
            # scores = F.softmax(F.relu(torch.einsum('bnl, bml -> bnm', q, k) - torch.einsum('bnl, bml -> bnm', k, q)) / math.sqrt(d), dim=-1) + unit

            scores = F.softmax(torch.bmm(q, k.transpose(1, 2))
                               / math.sqrt(d), dim=-1).reshape(B, self.out_var_dims, self.out_var_dims) + unit  # B, M, M
            # A = F.softmax(F.relu(scores * supports), dim=-1) + unit

            A = F.softmax(self.weights * scores + (1 - self.weights) * supports, dim=-1) + unit
            # A = F.softmax(scores, dim=-1) + unit

            x = torch.einsum('bnm, bmld -> bnld', A, self.w_v(x))
            x = self.w_o(x) + self.w_res(res_x)
        return x, scores, supports,self.weights


# if __name__ =='__main__':
#     x = torch.rand(128, 20, 36, 10) * 2 - 1
#     model = Attention_DAGCN(num_time_steps=36, num_nodes=20, embed_dim=5, in_dims=10, out_dims=64, is_Dimension_Reduction=True)
#     out, scores, supports = model(x)
#     print(out.shape)

In [ ]:
import math
import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal
# from 研究生课题.前沿研究.Chaotic_Net.models.DAGCN import DAGCN, Mulit_DAGCN
# from 研究生课题.前沿研究.Chaotic_Net.models.DAGCN import Attention_DAGCN

# from einops import rearrange
from typing import Union, Tuple
from torch.distributions.normal import Normal

import seaborn as sns
import matplotlib.pyplot as plt

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


# 生成旋转矩阵
def precompute_freqs_cis(atten_dim: int, seq_len: int, theta: float = 10000.0):
    theta_i_e = (torch.arange(0, atten_dim, 2)[:(atten_dim // 2)].float() / atten_dim) * -1
    freqs = theta ** theta_i_e.to(device)
    position = torch.arange(seq_len).to(device)  # [0, 1, 2, 3, ..., seq_len]
    freqs = torch.outer(position, freqs).float().to(device)  # 求向量的外积,维度为[seq_len,atten_dim]
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs).to(device)  # 将上一步的结果写成复数的形式,模是1幅角是freqs
    return freqs_cis.view(1, 1, seq_len, atten_dim // 2)


def apply_rope(q: torch.Tensor, k: torch.Tensor, rotate_vecs: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    作用: 将q,k向量分别与旋转向量相乘,得到旋转后的q,k向量q/k_rotated。然后进行点乘得到具有位置信息的attention分数
    输入: q->weight_q(input_vecs), k->weight_k(input_vecs), rotaed_vecs->旋转向量
    """
    q = q.contiguous()
    k = k.contiguous()

    # 计算过程q:[batch_size,atten_heads,seq_len,atten_dim]->q_complex:[b,a_h,s,a_d//2,2]->[b,a_h,s,a_d//2]->[b,a_h,s,a_d//2,2]
    q_complex = torch.view_as_complex(
        q.float().reshape(*q.shape[:-1], -1, 2))  # [batch_size,atten_heads,seq_len,atten_dim//2,2]
    k_complex = torch.view_as_complex(k.float().reshape(*k.shape[:-1], -1, 2))  # 将一个大小为n的向量两两组合形成复数来计算
    # 位置编码只和向量的序列位置还有向量本身有关，和batch以及注意力头无关，所以只用关注第二维和第四维

    q_rotated = torch.view_as_real(q_complex * rotate_vecs).flatten(
        3)  # 恢复成原来的样子，将第三维之后压平，也就是(atten_dim//2,2)->(atten_dim)
    k_rotated = torch.view_as_real(k_complex * rotate_vecs).flatten(3)
    return q_rotated.type_as(q), k_rotated.type_as(q)


class Channel_Embedding(nn.Module):
    def __init__(self, list_input_dim, num_experts=4, out_channels=10, k=4, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding, self).__init__()
        self.k = k
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels
        self.list_input_dim = list_input_dim

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        for input_dim in list_input_dim:
            experts = nn.Sequential(
                nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                          stride=emb_stride),
                nn.Tanh(),
                nn.Conv1d(in_channels=out_channels, out_channels=out_channels * num_experts, kernel_size=1)
            )
            # w_gate = nn.Parameter(0.005*torch.randn(input_dim * 5, num_experts), requires_grad=True).to(device)
            # w_noise = nn.Parameter(0.005*torch.randn(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
            w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

            self.list_m_experts.append(experts)
            self.list_m_gating.append(w_gate)
            self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, m, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[m]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[m]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_combine_features = []
        list_spatial_x = []
        list_gates = []
        for i, dim in enumerate(self.list_input_dim):
            input_x = x[:, index:index + dim, :]
            list_spatial_x.append(x[:, index, :].unsqueeze(1))
            index += dim

            B, d, L = input_x.shape
            # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
            gates, load = self.noisy_top_k_gating(input_x[:, :, -6:-1].reshape(B, dim * 5), i, self.training)
            list_gates.append(gates.unsqueeze(-1))

            # calculate importance loss
            importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
            loss = self.cv_squared(importance) + self.cv_squared(load)
            loss *= loss_coef
            out_loss = out_loss + loss

            out_raw = self.list_m_experts[i](input_x)  # B,D*E,l
            out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

            moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

            list_combine_features.append(moe_out.unsqueeze(1))
        combine_features = torch.cat(list_combine_features, dim=1)
        gates = torch.cat(list_gates, dim=-1)
        return combine_features, out_loss, gates


class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, seq_len, dropout=0.1, gate_mlp=False):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm((seq_len, d_in), eps=1e-6)
        self.dropout = nn.Dropout(dropout)
        self.gate_mlp = gate_mlp
        self.gate_linear = nn.Linear(d_in, d_hid)
        self.silu = nn.SiLU()

    def forward(self, x):
        residual = x

        if self.gate_mlp == False:
            x = self.w_2(F.relu(self.w_1(x)))

        else:
            x = self.w_2(F.relu(self.w_1(x) * (self.silu(self.gate_linear(x)))))

        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)
        return x


class MultiAttention(nn.Module):
    def __init__(self, input_dim, num_hiddens, num_heads, seq_len, dropout=0.1):
        super(MultiAttention, self).__init__()
        self.num_heads = num_heads
        self.num_hiddens = num_hiddens

        self.w_q = nn.Linear(input_dim, num_hiddens)
        self.w_k = nn.Linear(input_dim, num_hiddens)
        self.w_v = nn.Linear(input_dim, num_hiddens)

        self.w_o = nn.Linear(num_hiddens, num_hiddens, bias=False)

        self.drop_out = nn.Dropout(dropout)

        self.freqs_cis = precompute_freqs_cis(atten_dim=num_hiddens // num_heads, seq_len=seq_len)

    def transpose_qk_RoPE(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B,num_heads,查询数或者键值对数,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x

    def transpose_qkv(self, x, num_heads):
        '''
        :param x: shape(B,查询数或者键值对数,num_hiddens)
        :return:  shape(B*num_heads,查询数或者键值对数,num_heads,num_hiddens/num_heads)      # num_hiddens为num_heads的整数倍
        '''
        x = x.reshape(x.shape[0], x.shape[1], num_heads, -1)
        x = x.permute(0, 2, 1, 3)
        return x.reshape(-1, x.shape[2], x.shape[3])

    def transpose_output(self, x, num_heads):
        x = x.reshape(-1, num_heads, x.shape[1], x.shape[2])
        x = x.permute(0, 2, 1, 3)
        return x.reshape(x.shape[0], x.shape[1], -1)

    def forward(self, x):
        B, time_len, input_dim = x.shape
        q = self.transpose_qk_RoPE(self.w_q(x), self.num_heads)
        k = self.transpose_qk_RoPE(self.w_k(x), self.num_heads)  # B,heads,time_len,dim/heads
        xq, xk = apply_rope(q=q, k=k, rotate_vecs=self.freqs_cis)

        q = xq.reshape(-1, time_len, self.num_hiddens // self.num_heads)
        k = xk.reshape(-1, time_len, self.num_hiddens // self.num_heads)

        v = self.transpose_qkv(self.w_v(x), self.num_heads)

        d = x.shape[-1]
        att = F.softmax(torch.bmm(q, k.transpose(1, 2)), dim=-1)
        scores = att / math.sqrt(d)

        output = torch.bmm(self.drop_out(scores), v)
        output_concat = self.transpose_output(output, self.num_heads)
        return self.w_o(output_concat), att
 

class model_1(nn.Module):
    def __init__(self, M, list_input_dims, d=64, num_layers=1, num_experts=3, k=3,
                 pred_len=1, time_len = 32, is_Dimension_Reduction=False,out_var_dims = 3):
        super(model_1, self).__init__()

        if is_Dimension_Reduction == False:
            self.M = M
        else:
            self.M = out_var_dims
            self.w_res = nn.Conv2d(in_channels=M, out_channels=out_var_dims, kernel_size=1)

        self.pred_len = pred_len
        self.num_layers = num_layers
        self.list_input_dims = list_input_dims
        self.is_Dimension_Reduction = is_Dimension_Reduction
        self.embedding = Channel_Embedding(list_input_dim=self.list_input_dims, out_channels=d, 
                                           num_experts=num_experts, k=k)

        self.time_lens = time_len-2
        self.indenpendet_transformer = nn.ModuleList(
            [MultiAttention(input_dim=d, num_hiddens=d, num_heads=8, seq_len=self.time_lens).to(device) for _ in range(M)])
        self.list_ln = nn.ModuleList([nn.LayerNorm(normalized_shape=(self.time_lens, d)) for _ in range(M)])
        self.indenpendet_FFN = nn.ModuleList(
            [PositionwiseFeedForward(d, d, seq_len=self.time_lens, gate_mlp=True).to(device) for _ in range(M)])

        # self.dagcn = Mulit_DAGCN(num_time_steps=30, num_nodes=M, in_dims=d, out_dims=d, cheb_k=3, embed_dim=2)
        self.dagcn = Attention_DAGCN(num_time_steps=self.time_lens, num_nodes=M, embed_dim=5, 
                                     is_Dimension_Reduction=is_Dimension_Reduction,out_var_dims=out_var_dims)
        self.ln2 = nn.LayerNorm(normalized_shape=(self.time_lens, d))

        self.weights = nn.Parameter(torch.ones(1) / 2)  # 可学习的权重参数

        self.FFN = PositionwiseFeedForward(d, d, seq_len=self.time_lens)
        self.FFN2 = PositionwiseFeedForward(d, d, seq_len=self.time_lens)

        self.linear_time = nn.Linear(self.time_lens, pred_len)
        # self.linear_time = nn.Linear(pred_len, pred_len)
        self.linear_feat = nn.Linear(self.M * d, 1)
        # self.linear_feat = nn.Linear(self.M * d, 80)

    def forward(self, in_x):
        B, _, _ = in_x.shape
        x = in_x.permute(0, 2, 1)
        x, loss, gates = self.embedding(x)
        x = x.transpose(2, 3)  # B,M,L,D
        
        for _ in range(self.num_layers):
            list_multi_attention = []

            if self.is_Dimension_Reduction == False:
                list_independent_x = []
                for i in range(self.M):
                    in_x = x[:, i, :, :]

                    independent_x, attention_i = self.indenpendet_transformer[i](in_x)
                    independent_x = self.list_ln[i](in_x + independent_x)
                    independent_x = self.indenpendet_FFN[i](independent_x) + independent_x
                    list_independent_x.append(independent_x.unsqueeze(1))
                    list_multi_attention.append(attention_i.unsqueeze(1))
                    
                independent_x = torch.cat(list_independent_x, dim=1)
                independent_x = self.dagcn(independent_x.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)

                dagcn_x, scores, supports,weights = self.dagcn(independent_x + x)
                dagcn_x = self.FFN2(self.ln2(dagcn_x + x)) + x
                # dagcn_x = self.FFN2(dagcn_x) + x

            else:
                dagcn_x = x
                dagcn_x, scores, supports = self.dagcn(dagcn_x)
                dagcn_x = self.FFN2(self.ln2(dagcn_x)) + self.w_res(x)

                list_independent_x = []
                for i in range(self.M):
                    in_x = dagcn_x[:, i, :, :]

                    independent_x = self.indenpendet_transformer[i](in_x)
                    independent_x = self.list_ln[i](in_x + independent_x)
                    independent_x = self.indenpendet_FFN[i](independent_x) + independent_x
                    list_independent_x.append(independent_x.unsqueeze(1))

                dagcn_x = torch.cat(list_independent_x, dim=1) + self.w_res(x)
                
            multi_attention = torch.cat(list_multi_attention, dim=1)

            # x = self.FFN2(dagcn_x)+ x
            # x = independent_x + x

        # print(x.shape)
        out = self.linear_feat(dagcn_x.permute(0,2,1,3).reshape(B,self.time_lens,-1)).squeeze(-1)
        # out = out[:,-self.pred_len:]
        out = self.linear_time(out)


        # KS
        # out = self.linear_feat(dagcn_x.permute(0,2,1,3).reshape(B,self.time_lens,-1)).permute(0,2,1)
        # # out = self.linear_time(out).squeeze(-1)
        # out = out[:,:,-self.pred_len:].squeeze(-1)
        # print(gates)
        # print(weights)

        return out, loss, gates, _, _, multi_attention
        # return out, loss, gates, scores, supports, multi_attention


# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # B,L,D
#     # model = model_1(time_len=32,M=3,list_input_dims=[5,5,5])
#     model = model_1(time_len=32,M=5,list_input_dims=[5,4,1,3,2],is_Dimension_Reduction=True,num_layers=2,pred_len=6)

#     out, _, gates, scores, supports = model(x)
#     print(out.shape)


In [ ]:
### import time
import os
import time
import torch
import random
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import numpy as np

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 环境设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # 使用gpu
batch_size = 128
criterion = nn.MSELoss()

# 能固定dataloader的结果，使复现结果一致
def setup_seed(seed):
   torch.manual_seed(seed)
   torch.cuda.manual_seed_all(seed)
   np.random.seed(seed)
   random.seed(seed)
   torch.backends.cudnn.deterministic = True
setup_seed(20)

start_time = time.time()


def get_all_result(test_y, y_pred):
    test_y = np.nan_to_num(test_y)
    y_pred = np.nan_to_num(y_pred)
    mse = mean_squared_error(test_y,y_pred)
    rmse = mean_squared_error(test_y,y_pred)**0.5
    mae= mean_absolute_error(test_y,y_pred)
    mape = mean_absolute_percentage_error(test_y,y_pred)
    r2 = r2_score(test_y,y_pred)

    print(f'mse:{mse}, rmse:{rmse}, mae:{mae},mape:{mape},r2:{r2}')
    return mse, rmse, mae, mape, r2


def mean_squared_error(y_true, y_pred):
  # 确保输入数据长度一致
    if len(y_true) != len(y_pred):
        raise ValueError("长度不一致：y_true 和 y_pred")

      # 计算MSE
    squared_errors = [(y_true[i] - y_pred[i])**2 for i in range(len(y_true))]
    return sum(squared_errors) / len(y_true)


def train(model, optimizer, scheduler, epoch, train_dataloader, train_seq, type_loss):
    model.train()  # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for batch_index,batch_data in enumerate(train_dataloader):
        data,targets = batch_data       #7 torch.Size([B, L, D]) torch.Size([B, pred_len])
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        output, au_loss, gates, scores, supports, multi_attention = model(data)
        loss = criterion(output, targets)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # 防止梯度爆炸或梯度消失问题
        optimizer.step()
        total_loss += loss.item()
        log_interval = int(len(train_seq) / batch_size/5 )
        if batch_index % log_interval == 0 and batch_index > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            # print('| epoch {:3d} | {:5d}/{:5d} batches | '
            #       'lr {:02.6f} | {:5.2f} ms | '
            #       'loss {:5.5f}'.format(
            #     epoch, batch_index, len(train_seq) // batch_size, scheduler.get_lr()[0],
            #                   elapsed * 1000 / log_interval,cur_loss))  # , math.exp(cur_loss)
            total_loss = 0
            start_time = time.time()
    return model


def evaluate(eval_model, dataloader, data_source_x,type_loss):
    eval_model.eval()  # Turn on the evaluation mode
    total_loss = 0.
    result = torch.Tensor(0).to(device)
    truth = torch.Tensor(0).to(device)
    save_gates = torch.Tensor(0).to(device)
    save_multi_attention_scores = torch.Tensor(0).to(device)
    save_score = torch.Tensor(0).to(device)
    save_support = torch.Tensor(0).to(device)

    with torch.no_grad():
        for batch_index,dataset in enumerate(dataloader):
            data, targets = dataset
            data = data.to(device)
            targets = targets.to(device)
            output, au_loss, gates, scores, supports, multi_attention = eval_model(data)

            if type_loss == 'mse':
                loss = criterion(output, targets)
                total_loss +=  len(data)*loss.item()

            else:
                loss, loss_shape, loss_temporal = dilate_loss(output, targets, alpha=0.8, gamma=0.01, device=device)
                total_loss +=  len(data)*loss.item()

            result = torch.cat((result.cpu(), output.cpu()),0)  # todo: check this. -> looks good to me
            truth = torch.cat((truth.cpu(), targets.cpu()), 0)

            if batch_index == 0:
                # save_score = torch.cat((save_score.cpu(), scores.cpu()), 0)
                # save_support = torch.cat((save_support.cpu(), supports.cpu()), 0)
                save_gates = torch.cat((save_gates.cpu(), gates.cpu()), 0)
                save_multi_attention_scores = torch.cat((save_multi_attention_scores.cpu(), multi_attention.cpu()), 0)
            else:
                pass

    
            # save_score = torch.cat((save_score.cpu(), scores.cpu()), 0)
            # save_support = torch.cat((save_support.cpu(), supports.cpu()), 0)
            # save_gates = torch.cat((save_gates.cpu(), gates.cpu()), 0)
            # save_multi_attention_scores = torch.cat((save_multi_attention_scores.cpu(), multi_attention.cpu()), 0)
        
        return total_loss / len(data_source_x),result,truth, save_gates, save_score, save_support, save_multi_attention_scores



def get_model(type_name, model_name, load_name, wo4_type_model='MLP',wo5_type_model='DCMC',pred_len=1,time_len=32,num_experts=8,k=2):
    model = None        # 定义模型

    # wo1_SR
    if load_name == 'tr':
        if model_name == 'Chaotic-Net':
            if type_name == 'lorenz':
                model = model_1(time_len=time_len,M=3,list_input_dims=[5,5,5],d=64).to(device)
            if type_name == 'rossler':
                model = model_1(time_len=time_len,M=3,list_input_dims=[3,5,5],d=128,pred_len=pred_len,num_experts=4,k=4).to(device)
            if type_name == 'power':
                model = model_1(time_len=time_len,M=6,list_input_dims=[5,5,5,5,5,5],d=32).to(device)
            if type_name == 'lorenz-96':
                model = model_1(time_len=time_len,M=40,list_input_dims=[1]*40,d=32,
                                is_Dimension_Reduction=False,out_var_dims=8,num_experts=4,k=2).to(device)
            if type_name == 'KS':
                model = model_1(time_len=time_len,M=20,list_input_dims=[4]*20,d=32,
                                is_Dimension_Reduction=True,out_var_dims=3,num_experts=4,k=1,num_layers=1).to(device)
            
            if type_name == 'EMD_power':
                model = model_1(time_len=32,M=6,list_input_dims=[5,5,7,6,4,5],d=32).to(device)
            if type_name == 'EMD_lorenz':
                model = model_1(time_len=32,M=3,list_input_dims=[5,5,5],d=64).to(device)    
            if type_name == 'EMD_rossler':
                model = model_1(time_len=32,M=3,list_input_dims=[3,5,5],d=64).to(device) 
            
            if type_name == 'EMD_power2':
                model = model_1(time_len=32,M=6,list_input_dims=[9,9,1,9,10,10],d=32).to(device)
            if type_name == 'EMD_lorenz2':
                model = model_1(time_len=32,M=3,list_input_dims=[5,6,6],d=64).to(device)    
            if type_name == 'EMD_rossler2':
                model = model_1(time_len=32,M=3,list_input_dims=[5,5,6],d=32).to(device)
            
            # if type_name == 'Wave_power2':
            #     model = model_1(time_len=32,M=6,list_input_dims=[6,6,6,6,6,6],d=32).to(device)
            # if type_name == 'Wave_lorenz2':
            #     model = model_1(time_len=32,M=3,list_input_dims=[6,6,6],d=64).to(device)    
                
            if type_name == 'Wave_power':
                model = model_1(time_len=32,M=6,list_input_dims=[6,6,6,6,6,6],d=32).to(device)
            if type_name == 'Wave_lorenz':
                model = model_1(time_len=32,M=3,list_input_dims=[6,6,6],d=64).to(device)
            if type_name == 'Wave_rossler':
                model = model_1(time_len=32,M=3,list_input_dims=[6,6,6],d=32).to(device)
            
                        
        if model_name == 'wo2_MoEE':
            if type_name == 'rossler':
                model = wo2_MoEE_Model(time_len=time_len,M=3,list_input_dims=[3,5,5],d=64).to(device)
        if model_name == 'wo6_Expert_Linear':
            if type_name == 'rossler':
                model = wo6_Expert_Linear(time_len=time_len,M=3,list_input_dims=[3,5,5],d=64).to(device)
        if model_name == 'wo7_Expert_MLP':
            if type_name == 'rossler':
                model = wo7_Expert_MLP(time_len=32,M=3,list_input_dims=[3,5,5],d=64).to(device)

        # wo3_independent
        if model_name == 'wo3_independent':
            if type_name == 'rossler':
                model = wo3_independent_Model(time_len=32,M=3,list_input_dims=[3,5,5],d=64)
        if model_name == 'wo4_Transformer':
            if type_name == 'rossler':
                model = wo4_Transformer_Model(time_len=32,M=3,list_input_dims=[3,5,5],type_model=wo4_type_model,d=64)
        if model_name == 'wo5_DAGCN':
            if type_name == 'rossler':
                model = wo5_DCMC_Model(time_len=32,M=3,list_input_dims=[3,5,5],d=64,model_name=wo5_type_model)
                
    else:
        # Chaotic-Net
        if model_name == 'Chaotic-Net':
            if type_name == 'lorenz':
                model = model_1(time_len=time_len,M=3,list_input_dims=[5,5,5],d=128,num_layers=1,
                                num_experts=num_experts,k=k,pred_len=pred_len).to(device)
                
            if type_name == 'power':
                model = model_1(time_len=time_len,M=6,list_input_dims=[5,5,5,5,5,5],d=128,
                                num_experts=num_experts,k=k,pred_len=pred_len,
                                is_Dimension_Reduction=False).to(device)  
            
        # wo2_WoEE
        if model_name == 'wo2_MoEE':
            if type_name == 'lorenz':
                model = wo2_MoEE_Model(time_len=32,M=3,list_input_dims=[5,5,5],d=64)
            if type_name == 'power':
                model = wo2_MoEE_Model(time_len=32,M=6,list_input_dims=[5,5,5,5,5,5],d=32)
            
        # wo3_independent
        if model_name == 'wo3_independent':
            if type_name == 'lorenz':
                model = wo3_independent_Model(time_len=32,M=3,list_input_dims=[5,5,5],d=64)
            if type_name == 'power':
                model = wo3_independent_Model(time_len=32,M=6,list_input_dims=[5,5,5,5,5,5],d=32)

        # wo4_Transformer
        if model_name == 'wo4_Transformer':
            if type_name == 'lorenz':
                model = wo4_Transformer_Model(time_len=32,M=3,list_input_dims=[5,5,5],type_model=wo4_type_model,d=64)
            if type_name == 'power':
                model = wo4_Transformer_Model(time_len=32,M=6,list_input_dims=[5,5,5,5,5,5],type_model=wo4_type_model,d=32)

        # wo5_DAGCN
        if model_name == 'wo5_DAGCN':
            if type_name == 'lorenz':
                model = wo5_DCMC_Model(time_len=32,M=3,list_input_dims=[5,5,5],d=64,model_name=wo5_type_model)
            if type_name == 'power':
                model = wo5_DCMC_Model(time_len=32,M=6,list_input_dims=[5,5,5,5,5,5],d=32,model_name=wo5_type_model)

        # wo6_Expert_Linear
        if model_name == 'wo6_Expert_Linear':
            if type_name == 'lorenz':
                model = wo6_Expert_Linear(time_len=32,M=3,list_input_dims=[5,5,5],d=64)
            if type_name == 'power':
                model = wo6_Expert_Linear(time_len=32,M=6,list_input_dims=[5,5,5,5,5,5],d=32)
                
        # wo7_Expert_MLP
        if model_name == 'wo7_Expert_MLP':
            if type_name == 'lorenz':
                model = wo7_Expert_MLP(time_len=32,M=3,list_input_dims=[5,5,5],d=64)
            if type_name == 'power':
                model = wo7_Expert_MLP(time_len=32,M=6,list_input_dims=[5,5,5,5,5,5],d=32)
    return model


def read_npy(type_name,load_name,model_name,epochs,weight_decay,lr = 0.001,wo4_type_model='MLP',
             wo5_type_model='DCMC',seed=0,pred_len=1,time_len=32,num_experts=2,k=2):
    setup_seed(seed)
    if pred_len == 1:
        if load_name == 'sr':
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/sr_{}_{}.npy'.format(type_name,seed),allow_pickle=True).item()
        else:
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_{}_{}.npy'.format(type_name,seed),allow_pickle=True).item()
        # saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_KS_42.npy',allow_pickle=True).item()

    else:
        if load_name == 'sr':
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/sr_{}_{}_({}).npy'.format(type_name,seed,pred_len),allow_pickle=True).item()
        else:
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_{}_{}_({}).npy'.format(type_name,seed,pred_len),allow_pickle=True).item()

    # saved_dict = np.load(r'/kaggle/input/original-chaotic-data/sr_{}_({}_{}).npy'.format(type_name,time_len,pred_len),allow_pickle=True).item()
    
    
    train_seq = saved_dict['train']['train_seq'].to(device)
    train_label = saved_dict['train']['train_label'].to(device)
    train_dataloader = saved_dict['train']['train_dataloader']

    val_seq = saved_dict['val']['val_seq'].to(device)
    val_label = saved_dict['val']['val_label'].to(device)
    val_dataloader = saved_dict['val']['val_dataloader']

    test_seq = saved_dict['test']['test_seq'].to(device)
    test_label = saved_dict['test']['test_label'].to(device)
    test_dataloader = saved_dict['test']['test_dataloader']

    print('----------train--------')
    print(f'train X shape:{train_seq.shape}')
    print(f'train y shape:{train_label.shape}')

    print('----------val--------')
    print(f'val X shape:{val_seq.shape}')
    print(f'val y shape:{val_label.shape}')

    print('----------test--------')
    print(f'test X shape:{test_seq.shape}')
    print(f'test y shape:{test_label.shape}')
    
    model = get_model(type_name, model_name, load_name, wo4_type_model=wo4_type_model,wo5_type_model=wo5_type_model,
                      pred_len=pred_len,time_len=time_len,num_experts=num_experts,k=k).to(device)
    
    type_loss = 'mse'
    # type_loss = 'dilate'

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.98)       # 学习率调度器对象,调度器会在每个10个epoch后将学习率乘以gamma。
    best_test_loss = 1000
    best_val_output, best_val_target,best_test_output, best_test_target = 0, 0, 0, 0
    best_train_save_gates, best_test_save_gates, best_train_attention_scores, best_test_attention_scores = 0, 0, 0, 0
    best_train_save_scores, best_test_save_scores, best_train_save_supports, best_test_save_supports = 0, 0, 0, 0

    best_model = None
    list_train_loss, list_test_loss = [], []

    # 训练模型
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train_model = train(model, optimizer, scheduler, epoch,train_dataloader, train_seq,type_loss)

        train_loss, train_output, train_target,train_save_gates, train_save_score, train_save_support, train_multi_attention_scores = evaluate(train_model, train_dataloader, train_seq,type_loss)
        test_loss, test_output, test_target,test_save_gates, test_save_score, test_save_support, test_multi_attention_scores = evaluate(train_model, test_dataloader, test_seq,type_loss)
        list_train_loss.append(train_loss)
        list_test_loss.append(test_loss)

        if test_loss<best_test_loss:
            best_test_loss = test_loss
            best_test_output = test_output
            best_test_target = test_target
            best_model = train_model
            
            best_train_save_gates = train_save_gates
            best_test_save_gates = test_save_gates
            best_train_attention_scores = train_multi_attention_scores
            best_test_attention_scores = test_multi_attention_scores

            best_train_save_scores = train_save_score
            best_test_save_scores = test_save_score
            best_train_save_supports = train_save_support
            best_test_save_supports = test_save_support


        if epoch%10==0:
            print(
                '| end of epoch {:3d} | time: {:5.2f}s  | train loss {:.10f}| test loss {:.10f} '.format(
                    epoch, (time.time() - epoch_start_time), train_loss, test_loss))
            print('-' * 90)

        scheduler.step()        # 学习率调度器对象

    # data_out = pd.DataFrame({'true':[i[0] for i in best_test_target.tolist()],
    #                          'pred':[i[0] for i in best_test_output.tolist()]},
    #                         index=range(len(best_test_output)))
    
    # torch.save(best_model.state_dict(),r'/kaggle/working/best_model_{}.pth'.format(type_name))

    # print(data_out)
    # print(mean_squared_error(best_test_target,best_test_output))
    # print('train:',train_save_gates)
    
    # out_information = {
    #     'train_gates':best_train_save_gates,'test_gates':best_test_save_gates,
    #     'train_multi_attention_scores':best_train_attention_scores,'test_multi_attention_scores':best_test_attention_scores,
    #     'train_score':best_test_save_scores, 'test_score':best_test_save_scores, 
    #     'train_support':best_train_save_supports, 'test_support':best_test_save_supports, 
    #                   }
    # print(best_test_save_scores,best_test_save_scores.shape,best_test_save_supports)


    if type_name!='KS':
        if pred_len == 1:
            data_out = pd.DataFrame({'true':[i[0] for i in best_test_target.tolist()],
                                 'pred':[i[0] for i in best_test_output.tolist()]},
                                index=range(len(best_test_output)))
            
            get_all_result(data_out['true'], data_out['pred'])
            if model_name == 'wo4_Transformer':
                data_out.to_csv(r'/kaggle/working/{}_{}_{}_{}_{}.csv'.format(load_name,type_name,model_name,wo4_type_model,seed))
            if model_name == 'wo5_DAGCN':
                data_out.to_csv(r'/kaggle/working/{}_{}_{}_{}_{}.csv'.format(load_name,type_name,model_name,wo5_type_model,seed))
            else:
                folder_path = '/kaggle/working/new2/'
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                file_path = os.path.join(folder_path, '{}_{}_{}_MOEE_{}_{}.csv'.format(load_name,type_name,model_name,num_experts,k))
                print(file_path)
                data_out.to_csv(file_path)
                
                
        else:
            columns = ['true{}'.format(i+1) for i in range(pred_len)]
            columns.extend(['pred{}'.format(i+1) for i in range(pred_len)])
            true_df = pd.DataFrame(best_test_target.numpy(), columns=columns[:pred_len])
            pred_df = pd.DataFrame(best_test_output.numpy(), columns=columns[pred_len:])
            data_out = pd.concat([true_df, pred_df], axis=1)
            print(data_out)
            get_all_result(true_df, pred_df)
            
            data_out.to_csv(r'/kaggle/working/{}_{}_{}_{}_({}).csv'.format(load_name,type_name,model_name,seed,pred_len))
            # data_out.to_csv(r'/kaggle/working/{}_{}_{}_({}_{}).csv'.format(load_name,type_name,model_name,time_len,pred_len))

    else:
        columns = ['true{}'.format(i+1) for i in range(80)]
        columns.extend(['pred{}'.format(i+1) for i in range(80)])
        true_df = pd.DataFrame(best_test_target.numpy(), columns=columns[:80])
        pred_df = pd.DataFrame(best_test_output.numpy(), columns=columns[80:])
        data_out = pd.concat([true_df, pred_df], axis=1)
        print(data_out,111)
        
        get_all_result(true_df, pred_df)
        file_name = r'{}_{}_{}.csv'.format(load_name,type_name,model_name,time_len,pred_len)
        data_out.to_csv(file_name)
        

    # np.save(r'/kaggle/working/{}_{}_information.npy'.format(load_name,type_name), out_information)

    # plt.plot(best_test_target, label='真实值')
    # plt.plot(best_test_output, label='预测值')
    # plt.legend()
    # plt.show()


models = ['Chaotic-Net','wo2_MoEE','wo3_independent','wo4_Transformer','wo5_DAGCN', 'wo6_Expert_Linear', 'wo7_Expert_MLP']
load_name = 'sr'


# read_npy('power',load_name,models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=1,num_experts=8,k=6)
# read_npy('power',load_name,models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=1,num_experts=8,k=7)
# read_npy('power',load_name,models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=1,num_experts=8,k=8)


# for num in [8]:
#     for k in [1,2,3,4,5,6,7,8]:
#         if num<k:
#             continue
#         else:
#             print(num,k)
#             # read_npy('lorenz',load_name,models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=1,num_experts=num,k=k)
#             read_npy('power',load_name,models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=1,num_experts=num,k=k)




# read_npy('lorenz',load_name,models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=1)
# read_npy('rossler','tr',models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=1)
# read_npy('power',load_name,models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=1)


# for time_len in [8,16,24,32,40,48,56,64]:    
# for time_len in [40]:    
#     print(time_len,'lorenz')
#     read_npy('power','sr',models[0],epochs = 300, weight_decay=2,lr = 0.001,seed=42,pred_len=6,time_len=time_len)
    # read_npy('lorenz','sr',models[0],epochs = 300, weight_decay=0.02,lr = 0.005,seed=42,pred_len=6,time_len=time_len)



for pred_len in [10,11,12,4,5,6,7,8,9]:
    print(pred_len,'lorenz')
    # read_npy('power','sr',models[0],epochs = 3, weight_decay=0.5,lr = 0.001,seed=42,pred_len=pred_len)
    read_npy('rossler','tr',models[0],epochs = 300, weight_decay=0.02,lr = 0.005,seed=42,pred_len=pred_len)


# for i in [1]:
#     print(i,'rossler')
    # read_npy('lorenz-96','tr',models[0],epochs = 300, weight_decay=1.2,lr = 0.005,seed=i)
    # read_npy('KS','tr',models[0],epochs = 300, weight_decay=1.2,lr = 0.005,seed=42)

    # read_npy('rossler','tr',models[2],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)
    # read_npy('power','sr',models[2],epochs = 300, weight_decay=0.5,lr = 0.001,seed=i)


    # read_npy('power','sr',models[4], epochs = 300, weight_decay=0.5, lr = 0.001, seed=i, wo5_type_model='AGCN')
    # read_npy('power','sr',models[4], epochs = 300, weight_decay=0.5, lr = 0.001, seed=i, wo5_type_model='Attention')
    # read_npy('power','sr',models[4], epochs = 300, weight_decay=0.5, lr = 0.001, seed=i, wo5_type_model='DMCM')


    # read_npy('rossler','tr',models[4], epochs = 300, weight_decay=0.02, lr = 0.001, seed=i, wo5_type_model='AGCN')
    # read_npy('rossler','tr',models[4], epochs = 300, weight_decay=0.02, lr = 0.001, seed=i, wo5_type_model='Attention')
    # read_npy('rossler','tr',models[4], epochs = 300, weight_decay=0.02, lr = 0.001, seed=i, wo5_type_model='DMCM')


    # read_npy('lorenz','sr',models[3],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i,wo4_type_model='TCN')
    # read_npy('lorenz','sr',models[3],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i,wo4_type_model='LSTM')

    # read_npy('power','sr',models[3],epochs = 300, weight_decay=0.5,lr = 0.001,seed=i,wo4_type_model='TCN')
    # read_npy('power','sr',models[3],epochs = 300, weight_decay=0.5,lr = 0.001,seed=i,wo4_type_model='LSTM')

    # read_npy('rossler','tr',models[-2],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)
    # read_npy('rossler','tr',models[-1],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)
    # read_npy('Wave_rossler','tr',models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)




10 lorenz
----------train--------
train X shape:torch.Size([2550, 32, 13])
train y shape:torch.Size([2550, 10])
----------val--------
val X shape:torch.Size([450, 32, 13])
val y shape:torch.Size([450, 10])
----------test--------
test X shape:torch.Size([876, 32, 13])
test y shape:torch.Size([876, 10])
| end of epoch  10 | time:  0.71s  | train loss 0.0006753029| test loss 0.0006284631 
------------------------------------------------------------------------------------------
| end of epoch  20 | time:  0.75s  | train loss 0.0003694666| test loss 0.0003166221 
------------------------------------------------------------------------------------------


In [ ]:

# for i in [1,2,3,4,5]:
#     print(i,'Wave_lorenz2')
#     read_npy('Wave_lorenz2','tr',models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)


# for i in [1,2,3,4,5]:
#     print(i,'EMD_lorenz2')
#     read_npy('EMD_lorenz2','tr',models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)


# for i in [2,3,4,5]:
#     print(i,'Wave_power')
#     read_npy('Wave_power','tr',models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=i)


# for i in [1,2,3,4,5]:
#     print(i,'EMD_power2')
#     read_npy('EMD_power2','tr',models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=i)


# for i in [1,2,3,4,5]:
#     print(i,'EMD_power')
#     read_npy('EMD_power','tr',models[0],epochs = 300, weight_decay=0.5,lr = 0.001,seed=i)


# for i in [1,2,3,4,5]:
#     print(i,'EMD_lorenz')
#     read_npy('EMD_lorenz','tr',models[0],epochs = 300, weight_decay=0.02,lr = 0.001,seed=i)

In [22]:
import math
import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
from typing import Union, Tuple
from torch.distributions.normal import Normal
# from 研究生课题.前沿研究.Chaotic_Net.models.DAGCN import DAGCN, Mulit_DAGCN
# from 研究生课题.前沿研究.Chaotic_Net.models.DAGCN import Attention_DAGCN

# from einops import rearrange
from typing import Union, Tuple
from torch.distributions.normal import Normal

torch.manual_seed(21)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用gpu


class Channel_Embedding_ablation(nn.Module):
    def __init__(self, input_dim, num_experts=4, out_channels=10, k=4, emb_kernel_size=3, emb_stride=1):
        super(Channel_Embedding_ablation, self).__init__()
        self.k = k
        self.noisy_gating = True
        self.num_experts = num_experts
        self.out_channels = out_channels

        self.list_m_experts = nn.ModuleList()
        self.list_m_gating = []
        self.list_m_noise = []

        experts = nn.Sequential(
            nn.Conv1d(in_channels=input_dim, out_channels=out_channels, kernel_size=emb_kernel_size,
                      stride=emb_stride),
            nn.Tanh(),
            nn.Conv1d(in_channels=out_channels, out_channels=out_channels * num_experts, kernel_size=1)
        )
        # w_gate = nn.Parameter(0.005*torch.randn(input_dim * 5, num_experts), requires_grad=True).to(device)
        # w_noise = nn.Parameter(0.005*torch.randn(input_dim * 5, num_experts), requires_grad=True).to(device)
        w_gate = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)
        w_noise = nn.Parameter(torch.zeros(input_dim * 5, num_experts), requires_grad=True).to(device)

        self.list_m_experts.append(experts)
        self.list_m_gating.append(w_gate)
        self.list_m_noise.append(w_noise)

        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        """计算样本变异系数
        The squared coefficient of variation of a sample.
        Useful as a loss to encourage a positive distribution to be more uniform.
        Epsilons added for numerical stability.
        Returns 0 for an empty Tensor.
        Args:
        x: a `Tensor`.
        Returns:
        a `Scalar`(标量).
        """
        eps = 1e-10
        # if only num_experts = 1

        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        """Compute the true load per expert, given the gates.
        The load is the number of examples for which the corresponding gate is >0.
        Args:
        gates: a `Tensor` of shape [batch_size, n]
        Returns:
        a float32 `Tensor` of shape [n]
        """
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        """Helper function to NoisyTopKGating.
        Computes the probability that value is in top k, given different random noise.
        This gives us a way of backpropagating from a loss that balances the number
        of times each expert is in the top k experts per example.
        In the case of no noise, pass in None for noise_stddev, and the result will
        not be differentiable.
        Args:
        clean_values: a `Tensor` of shape [batch, n].
        noisy_values: a `Tensor` of shape [batch, n].  Equal to clean values plus
          normally distributed noise with standard deviation noise_stddev.
        noise_stddev: a `Tensor` of shape [batch, n], or None
        noisy_top_values: a `Tensor` of shape [batch, m].
           "values" Output of tf.top_k(noisy_top_values, m).  m >= k+1
        Returns:
        a `Tensor` of shape [batch, n].
        """
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        # is each value currently in the top k.
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def noisy_top_k_gating(self, x, train, noise_epsilon=1e-2):
        """Noisy top-k gating.
          See paper: https://arxiv.org/abs/1701.06538.
          Args:
            x: input Tensor with shape [batch_size, input_size]
            train: a boolean - we only add noise at training time.
            noise_epsilon: a float
          Returns:
            gates: a Tensor with shape [batch_size, num_experts]
            load: a Tensor with shape [num_experts]
        """
        clean_logits = x @ self.list_m_gating[0]  # 计算每个expert的权重
        if self.noisy_gating and train:  # 在训练中加入残差等
            raw_noise_stddev = x @ self.list_m_noise[0]  # 根据输入数据设置噪声权重
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits

        # calculate topk + 1 that will be needed for the noisy gates
        logits = self.softmax(logits)
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)
        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = top_k_logits / (top_k_logits.sum(1, keepdim=True) + 1e-6)  # normalization

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        index, out_loss = 0, 0
        list_gates = []

        B, d, L = x.shape
        # gates, load = self.noisy_top_k_gating(input_x[:,:,-1], i, self.training)
        gates, load = self.noisy_top_k_gating(x[:, :, -6:-1].reshape(B, d*5), self.training)
        list_gates.append(gates.unsqueeze(-1))

        # calculate importance loss
        importance = gates.sum(0)  # 将每个expert的gates权重加和,计算总的贡献值
        loss = self.cv_squared(importance) + self.cv_squared(load)
        loss *= loss_coef
        out_loss = out_loss + loss

        out_raw = self.list_m_experts[0](x)  # B,D*E,l
        out_raw = out_raw.permute(0, 2, 1).reshape(B, -1, self.out_channels, self.num_experts)  # B,l,D,E

        moe_out = torch.einsum("BLDE,BE->BLD", out_raw, gates).permute(0, 2, 1)

        return moe_out


# if __name__ == "__main__":
#         x = torch.randn((128, 15, 32))
#         model = Channel_Embedding_ablation(input_dim=15)
#         out = model(x)
#         print(out.shape)


# 对比实验

## LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class LSTM_Model(nn.Module):
    def __init__(self, seq_len, input_dim, hidden_dim1, hidden_dim2, pred_len,num_experts=2,k=2):
        # def __init__(self, seq_len, input_dim, hidden_dim1, hidden_dim2, pred_len):
        super(LSTM_Model, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        # self.dropout2 = nn.Dropout(0.2)
        self.lstm3 = nn.LSTM(hidden_dim2, hidden_dim2, batch_first=True)
        self.fc_feature = nn.Linear(hidden_dim2, 1)
        # self.fc_feature = nn.Linear(hidden_dim2, 80)

        self.fc_time = nn.Linear(seq_len, pred_len)

        self.embedding = Channel_Embedding_ablation(input_dim, out_channels=input_dim, num_experts=num_experts, k=k)

    def forward(self, x):
        B, _, _ = x.shape
        x = x.permute(0, 2, 1)
        x = self.embedding(x)
        x = x.transpose(1, 2)  # B,M,L,D

        # x = x.permute(0,2,1)
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        # x = self.dropout2(x) # Uncomment if needed
        x, _ = self.lstm3(x)
        # Flatten the output for the Dense layer

        x = self.fc_feature(x).squeeze(-1)
        x = self.fc_time(x)

        # x = self.fc_feature(x).permute(0,2,1)
        # x = self.fc_time(x).squeeze(-1)
        return x


# if __name__ == "__main__":
#     x = torch.randn((128,32,15)).to(devide)        # [B, L, D]
#     model = LSTM_Model(seq_len=30,input_dim=15, hidden_dim1=128, hidden_dim2=64, pred_len=1)
#     out = model(x)
#     print(out.shape)


## Transformer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvLayer(nn.Module):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = nn.Conv1d(in_channels=c_in,
                                  out_channels=c_in,
                                  kernel_size=3,
                                  padding=2,
                                  padding_mode='circular')
        self.norm = nn.BatchNorm1d(c_in)
        self.activation = nn.ELU()
        self.maxPool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.downConv(x.permute(0, 2, 1))
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        x = x.transpose(1, 2)
        return x


class EncoderLayer(nn.Module):
    def __init__(self, attention, d_model, d_ff=None, dropout=0.1, activation="relu"):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.attention = attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        new_x, attn = self.attention(
            x, x, x,
            attn_mask=attn_mask,
            tau=tau, delta=delta
        )
        x = x + self.dropout(new_x)

        y = x = self.norm1(x)
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))

        return self.norm2(x + y), attn


class Encoder(nn.Module):
    def __init__(self, attn_layers, conv_layers=None, norm_layer=None):
        super(Encoder, self).__init__()
        self.attn_layers = nn.ModuleList(attn_layers)
        self.conv_layers = nn.ModuleList(conv_layers) if conv_layers is not None else None
        self.norm = norm_layer

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        # x [B, L, D]
        attns = []
        if self.conv_layers is not None:
            for i, (attn_layer, conv_layer) in enumerate(zip(self.attn_layers, self.conv_layers)):
                delta = delta if i == 0 else None
                x, attn = attn_layer(x, attn_mask=attn_mask, tau=tau, delta=delta)
                x = conv_layer(x)
                attns.append(attn)
            x, attn = self.attn_layers[-1](x, tau=tau, delta=None)
            attns.append(attn)
        else:
            for attn_layer in self.attn_layers:
                x, attn = attn_layer(x, attn_mask=attn_mask, tau=tau, delta=delta)
                attns.append(attn)

        if self.norm is not None:
            x = self.norm(x)

        return x, attns


class DecoderLayer(nn.Module):
    def __init__(self, self_attention, cross_attention, d_model, d_ff=None,
                 dropout=0.1, activation="relu"):
        super(DecoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.self_attention = self_attention
        self.cross_attention = cross_attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, cross, x_mask=None, cross_mask=None, tau=None, delta=None):
        x = x + self.dropout(self.self_attention(
            x, x, x,
            attn_mask=x_mask,
            tau=tau, delta=None
        )[0])
        x = self.norm1(x)

        x = x + self.dropout(self.cross_attention(
            x, cross, cross,
            attn_mask=cross_mask,
            tau=tau, delta=delta
        )[0])

        y = x = self.norm2(x)
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))

        return self.norm3(x + y)


class Decoder(nn.Module):
    def __init__(self, layers, norm_layer=None, projection=None):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList(layers)
        self.norm = norm_layer
        self.projection = projection

    def forward(self, x, cross, x_mask=None, cross_mask=None, tau=None, delta=None):
        for layer in self.layers:
            x = layer(x, cross, x_mask=x_mask, cross_mask=cross_mask, tau=tau, delta=delta)

        if self.norm is not None:
            x = self.norm(x)

        if self.projection is not None:
            x = self.projection(x)
        return x



import torch
class TriangularCausalMask():
    def __init__(self, B, L, device="cpu"):
        mask_shape = [B, 1, L, L]
        with torch.no_grad():
            self._mask = torch.triu(torch.ones(mask_shape, dtype=torch.bool), diagonal=1).to(device)

    @property
    def mask(self):
        return self._mask


class ProbMask():
    def __init__(self, B, H, L, index, scores, device="cpu"):
        _mask = torch.ones(L, scores.shape[-1], dtype=torch.bool).to(device).triu(1)
        _mask_ex = _mask[None, None, :].expand(B, H, L, scores.shape[-1])
        indicator = _mask_ex[torch.arange(B)[:, None, None],
                    torch.arange(H)[None, :, None],
                    index, :].to(device)
        self._mask = indicator.view(scores.shape).to(device)

    @property
    def mask(self):
        return self._mask


import torch
import torch.nn as nn
import numpy as np
from math import sqrt
from reformer_pytorch import LSHSelfAttention
from einops import rearrange, repeat


class DSAttention(nn.Module):
    '''De-stationary Attention'''

    def __init__(self, mask_flag=True, factor=5, scale=None, attention_dropout=0.1, output_attention=False):
        super(DSAttention, self).__init__()
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, H, E = queries.shape
        _, S, _, D = values.shape
        scale = self.scale or 1. / sqrt(E)

        tau = 1.0 if tau is None else tau.unsqueeze(
            1).unsqueeze(1)  # B x 1 x 1 x 1
        delta = 0.0 if delta is None else delta.unsqueeze(
            1).unsqueeze(1)  # B x 1 x 1 x S

        # De-stationary Attention, rescaling pre-softmax score with learned de-stationary factors
        scores = torch.einsum("blhe,bshe->bhls", queries, keys) * tau + delta

        if self.mask_flag:
            if attn_mask is None:
                attn_mask = TriangularCausalMask(B, L, device=queries.device)

            scores.masked_fill_(attn_mask.mask, -np.inf)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        V = torch.einsum("bhls,bshd->blhd", A, values)

        if self.output_attention:
            return V.contiguous(), A
        else:
            return V.contiguous(), None


class FullAttention(nn.Module):
    def __init__(self, mask_flag=True, factor=5, scale=None, attention_dropout=0.1, output_attention=False):
        super(FullAttention, self).__init__()
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, H, E = queries.shape
        _, S, _, D = values.shape
        scale = self.scale or 1. / sqrt(E)

        scores = torch.einsum("blhe,bshe->bhls", queries, keys)

        if self.mask_flag:
            if attn_mask is None:
                attn_mask = TriangularCausalMask(B, L, device=queries.device)

            scores.masked_fill_(attn_mask.mask, -np.inf)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        V = torch.einsum("bhls,bshd->blhd", A, values)

        if self.output_attention:
            return V.contiguous(), A
        else:
            return V.contiguous(), None


class ProbAttention(nn.Module):
    def __init__(self, mask_flag=True, factor=5, scale=None, attention_dropout=0.1, output_attention=False):
        super(ProbAttention, self).__init__()
        self.factor = factor
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)

    def _prob_QK(self, Q, K, sample_k, n_top):  # n_top: c*ln(L_q)
        # Q [B, H, L, D]
        B, H, L_K, E = K.shape
        _, _, L_Q, _ = Q.shape

        # calculate the sampled Q_K
        K_expand = K.unsqueeze(-3).expand(B, H, L_Q, L_K, E)
        # real U = U_part(factor*ln(L_k))*L_q
        index_sample = torch.randint(L_K, (L_Q, sample_k))
        K_sample = K_expand[:, :, torch.arange(
            L_Q).unsqueeze(1), index_sample, :]
        Q_K_sample = torch.matmul(
            Q.unsqueeze(-2), K_sample.transpose(-2, -1)).squeeze()

        # find the Top_k query with sparisty measurement
        M = Q_K_sample.max(-1)[0] - torch.div(Q_K_sample.sum(-1), L_K)
        M_top = M.topk(n_top, sorted=False)[1]

        # use the reduced Q to calculate Q_K
        Q_reduce = Q[torch.arange(B)[:, None, None],
                   torch.arange(H)[None, :, None],
                   M_top, :]  # factor*ln(L_q)
        Q_K = torch.matmul(Q_reduce, K.transpose(-2, -1))  # factor*ln(L_q)*L_k

        return Q_K, M_top

    def _get_initial_context(self, V, L_Q):
        B, H, L_V, D = V.shape
        if not self.mask_flag:
            # V_sum = V.sum(dim=-2)
            V_sum = V.mean(dim=-2)
            contex = V_sum.unsqueeze(-2).expand(B, H,
                                                L_Q, V_sum.shape[-1]).clone()
        else:  # use mask
            # requires that L_Q == L_V, i.e. for self-attention only
            assert (L_Q == L_V)
            contex = V.cumsum(dim=-2)
        return contex

    def _update_context(self, context_in, V, scores, index, L_Q, attn_mask):
        B, H, L_V, D = V.shape

        if self.mask_flag:
            attn_mask = ProbMask(B, H, L_Q, index, scores, device=V.device)
            scores.masked_fill_(attn_mask.mask, -np.inf)

        attn = torch.softmax(scores, dim=-1)  # nn.Softmax(dim=-1)(scores)

        context_in[torch.arange(B)[:, None, None],
        torch.arange(H)[None, :, None],
        index, :] = torch.matmul(attn, V).type_as(context_in)
        if self.output_attention:
            attns = (torch.ones([B, H, L_V, L_V]) /
                     L_V).type_as(attn).to(attn.device)
            attns[torch.arange(B)[:, None, None], torch.arange(H)[
                                                  None, :, None], index, :] = attn
            return context_in, attns
        else:
            return context_in, None

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L_Q, H, D = queries.shape
        _, L_K, _, _ = keys.shape

        queries = queries.transpose(2, 1)
        keys = keys.transpose(2, 1)
        values = values.transpose(2, 1)

        U_part = self.factor * \
                 np.ceil(np.log(L_K)).astype('int').item()  # c*ln(L_k)
        u = self.factor * \
            np.ceil(np.log(L_Q)).astype('int').item()  # c*ln(L_q)

        U_part = U_part if U_part < L_K else L_K
        u = u if u < L_Q else L_Q

        scores_top, index = self._prob_QK(
            queries, keys, sample_k=U_part, n_top=u)

        # add scale factor
        scale = self.scale or 1. / sqrt(D)
        if scale is not None:
            scores_top = scores_top * scale
        # get the context
        context = self._get_initial_context(values, L_Q)
        # update the context with selected top_k queries
        context, attn = self._update_context(
            context, values, scores_top, index, L_Q, attn_mask)

        return context.contiguous(), attn


class AttentionLayer(nn.Module):
    def __init__(self, attention, d_model, n_heads, d_keys=None,
                 d_values=None):
        super(AttentionLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)
        d_values = d_values or (d_model // n_heads)

        self.inner_attention = attention
        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_model, d_keys * n_heads)
        self.value_projection = nn.Linear(d_model, d_values * n_heads)
        self.out_projection = nn.Linear(d_values * n_heads, d_model)
        self.n_heads = n_heads

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, _ = queries.shape
        _, S, _ = keys.shape
        H = self.n_heads

        queries = self.query_projection(queries).view(B, L, H, -1)
        keys = self.key_projection(keys).view(B, S, H, -1)
        values = self.value_projection(values).view(B, S, H, -1)

        out, attn = self.inner_attention(
            queries,
            keys,
            values,
            attn_mask,
            tau=tau,
            delta=delta
        )
        out = out.view(B, L, -1)

        return self.out_projection(out), attn


class ReformerLayer(nn.Module):
    def __init__(self, attention, d_model, n_heads, d_keys=None,
                 d_values=None, causal=False, bucket_size=4, n_hashes=4):
        super().__init__()
        self.bucket_size = bucket_size
        self.attn = LSHSelfAttention(
            dim=d_model,
            heads=n_heads,
            bucket_size=bucket_size,
            n_hashes=n_hashes,
            causal=causal
        )

    def fit_length(self, queries):
        # inside reformer: assert N % (bucket_size * 2) == 0
        B, N, C = queries.shape
        if N % (self.bucket_size * 2) == 0:
            return queries
        else:
            # fill the time series
            fill_len = (self.bucket_size * 2) - (N % (self.bucket_size * 2))
            return torch.cat([queries, torch.zeros([B, fill_len, C]).to(queries.device)], dim=1)

    def forward(self, queries, keys, values, attn_mask, tau, delta):
        # in Reformer: defalut queries=keys
        B, N, C = queries.shape
        queries = self.attn(self.fit_length(queries))[:, :N, :]
        return queries, None


class TwoStageAttentionLayer(nn.Module):
    '''
    The Two Stage Attention (TSA) Layer
    input/output shape: [batch_size, Data_dim(D), Seg_num(L), d_model]
    '''

    def __init__(self, configs,
                 seg_num, factor, d_model, n_heads, d_ff=None, dropout=0.1):
        super(TwoStageAttentionLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.time_attention = AttentionLayer(FullAttention(False, configs.factor, attention_dropout=configs.dropout,
                                                           output_attention=configs.output_attention), d_model, n_heads)
        self.dim_sender = AttentionLayer(FullAttention(False, configs.factor, attention_dropout=configs.dropout,
                                                       output_attention=configs.output_attention), d_model, n_heads)
        self.dim_receiver = AttentionLayer(FullAttention(False, configs.factor, attention_dropout=configs.dropout,
                                                         output_attention=configs.output_attention), d_model, n_heads)
        self.router = nn.Parameter(torch.randn(seg_num, factor, d_model))

        self.dropout = nn.Dropout(dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.norm4 = nn.LayerNorm(d_model)

        self.MLP1 = nn.Sequential(nn.Linear(d_model, d_ff),
                                  nn.GELU(),
                                  nn.Linear(d_ff, d_model))
        self.MLP2 = nn.Sequential(nn.Linear(d_model, d_ff),
                                  nn.GELU(),
                                  nn.Linear(d_ff, d_model))

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        # Cross Time Stage: Directly apply MSA to each dimension
        batch = x.shape[0]
        time_in = rearrange(x, 'b ts_d seg_num d_model -> (b ts_d) seg_num d_model')
        time_enc, attn = self.time_attention(
            time_in, time_in, time_in, attn_mask=None, tau=None, delta=None
        )
        dim_in = time_in + self.dropout(time_enc)
        dim_in = self.norm1(dim_in)
        dim_in = dim_in + self.dropout(self.MLP1(dim_in))
        dim_in = self.norm2(dim_in)

        # Cross Dimension Stage: use a small set of learnable vectors to aggregate and distribute messages to build the D-to-D connection
        dim_send = rearrange(dim_in, '(b ts_d) seg_num d_model -> (b seg_num) ts_d d_model', b=batch)
        batch_router = repeat(self.router, 'seg_num factor d_model -> (repeat seg_num) factor d_model', repeat=batch)
        dim_buffer, attn = self.dim_sender(batch_router, dim_send, dim_send, attn_mask=None, tau=None, delta=None)
        dim_receive, attn = self.dim_receiver(dim_send, dim_buffer, dim_buffer, attn_mask=None, tau=None, delta=None)
        dim_enc = dim_send + self.dropout(dim_receive)
        dim_enc = self.norm3(dim_enc)
        dim_enc = dim_enc + self.dropout(self.MLP2(dim_enc))
        dim_enc = self.norm4(dim_enc)

        final_out = rearrange(dim_enc, '(b seg_num) ts_d d_model -> b ts_d seg_num d_model', b=batch)

        return final_out


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import math


class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()


class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()
        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(
            self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x


class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6,
                    'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x) + self.position_embedding(x)
        else:
            x = self.value_embedding(
                x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)


class DataEmbedding_inverted(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding_inverted, self).__init__()
        self.value_embedding = nn.Linear(c_in, d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        x = x.permute(0, 2, 1)
        # x: [Batch Variate Time]
        if x_mark is None:
            x = self.value_embedding(x)
        else:
            x = self.value_embedding(torch.cat([x, x_mark.permute(0, 2, 1)], 1))
        # x: [Batch Variate d_model]
        return self.dropout(x)


class DataEmbedding_wo_pos(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding_wo_pos, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x)
        else:
            x = self.value_embedding(x) + self.temporal_embedding(x_mark)
        return self.dropout(x)


class PatchEmbedding(nn.Module):
    def __init__(self, d_model, patch_len, stride, padding, dropout):
        super(PatchEmbedding, self).__init__()
        # Patching
        self.patch_len = patch_len
        self.stride = stride
        self.padding_patch_layer = nn.ReplicationPad1d((0, padding))

        # Backbone, Input encoding: projection of feature vectors onto a d-dim vector space
        self.value_embedding = nn.Linear(patch_len, d_model, bias=False)

        # Positional embedding
        self.position_embedding = PositionalEmbedding(d_model)

        # Residual dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # do patching
        n_vars = x.shape[1]
        x = self.padding_patch_layer(x)
        x = x.unfold(dimension=-1, size=self.patch_len, step=self.stride)
        x = torch.reshape(x, (x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
        # Input encoding
        x = self.value_embedding(x) + self.position_embedding(x)
        return self.dropout(x), n_vars


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class Transformer_Model(nn.Module):
    """
    Vanilla Transformer
    with O(L^2) complexity
    Paper link: https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
    """

    def __init__(self,seq_len,pred_len,enc_in, num_experts=2, k=2):
        super(Transformer_Model, self).__init__()
        self.pred_len = pred_len
        e_layers = 3
        n_heads = 8
        d_model = 64
        # Embedding
        self.enc_embedding = DataEmbedding(enc_in, d_model)

        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        FullAttention(False), d_model, n_heads),
                    d_model) for l in range(e_layers)
            ],
            norm_layer=torch.nn.LayerNorm(d_model)
        )
        # Decoder
        self.dense_feature = nn.Linear(d_model,1)
        # self.dense_feature = nn.Linear(d_model,80)
        
        self.embedding = Channel_Embedding_ablation(enc_in, out_channels=enc_in, num_experts=num_experts, k=k)


    def forecast(self, x_enc):
        # Embedding
        x_enc = self.enc_embedding(x_enc, None)      # torch.Size([128, 50, 32])
        enc_out, attns = self.encoder(x_enc, attn_mask=None)   # torch.Size([128, 50, 32])
        
        enc_out = self.dense_feature(enc_out)
        return enc_out


    def forward(self, x_enc):
        B, _, _ = x_enc.shape
        x = x_enc.permute(0, 2, 1)
        x = self.embedding(x)
        x_enc = x.transpose(1, 2)  # B,M,L,D

        
        dec_out = self.forecast(x_enc)
        
        return dec_out[:, -self.pred_len:, :].squeeze(-1)  # [B, L, D]
        # return dec_out[:, -self.pred_len:, :].squeeze(1)  # [B, L, D]


# if __name__ == "__main__":
#     x = torch.randn((128,50,15))        # [B, L, D]
#     model = Transformer_Model(seq_len=50,pred_len=1,enc_in=15)
#     out = model(x)
#     print(out.shape)



## DLinear

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """

    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """

    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean


class DLinear_Model(nn.Module):
    """
    Paper link: https://arxiv.org/pdf/2205.13504.pdf
    """

    def __init__(self,seq_len, pred_len, enc_in,decompsition_kernel_size=3, individual=False, num_experts=2, k=2):
        super(DLinear_Model, self).__init__()
        self.seq_len = seq_len          # 输入序列长度
        self.pred_len = pred_len        # 输出序列长度
        # Series decomposition block from Autoformer
        self.decompsition = series_decomp(decompsition_kernel_size)     # 使用平均池化达到切分季节和趋势的目的
        self.individual = individual        # True代表通道独立性，反之不使用通道独立性
        self.channels = enc_in            # 选择通道个数

        # 使用通道独立性，对每一个通道都使用不同的Linear进行预测，然后将预测结果统一
        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()

            for i in range(self.channels):
                self.Linear_Seasonal.append(
                    nn.Linear(self.seq_len, self.pred_len))
                self.Linear_Trend.append(
                    nn.Linear(self.seq_len, self.pred_len))

                self.Linear_Seasonal[i].weight = nn.Parameter(
                    (1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))
                self.Linear_Trend[i].weight = nn.Parameter(
                    (1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))

        # 不使用通道独立性，考虑到了通道之间的关系
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len, self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len, self.pred_len)

            self.Linear_Seasonal.weight = nn.Parameter(
                (1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))
            self.Linear_Trend.weight = nn.Parameter(
                (1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))

        self.dense_feature = nn.Linear(enc_in,1)
        # self.dense_feature = nn.Linear(enc_in,80)
        
        self.dense_time = nn.Linear(self.seq_len,pred_len)


        self.embedding = Channel_Embedding_ablation(enc_in, out_channels=enc_in, num_experts=num_experts, k=k)



    def encoder(self, x):
        seasonal_init, trend_init = self.decompsition(x)        # x: B, L(seq_len), D(多变量个数)
        seasonal_init, trend_init = seasonal_init.permute(0, 2, 1), trend_init.permute(0, 2, 1)     # B, D, L

        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.pred_len],
                                          dtype=seasonal_init.dtype).to(seasonal_init.device)       # B, D, pred_len
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.pred_len],
                                       dtype=trend_init.dtype).to(trend_init.device)                # B, D, pred_len
            for i in range(self.channels):
                seasonal_output[:, i, :] = self.Linear_Seasonal[i](
                    seasonal_init[:, i, :])
                trend_output[:, i, :] = self.Linear_Trend[i](
                    trend_init[:, i, :])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)
        x = seasonal_output + trend_output
        return x.permute(0, 2, 1)

    def forward(self, x):
        B, _, _ = x.shape
        x = x.permute(0, 2, 1)
        x = self.embedding(x)
        x = x.transpose(1, 2)  # B,M,L,D

        
        dec_out = self.encoder(x)
        # print(dec_out.shape)

        dec_out = self.dense_feature(dec_out).squeeze(-1)
        # print(dec_out.shape)
        # out = self.dense_time(dec_out)
        # return out  # [B, L, D]

        return dec_out[:, -self.pred_len:]  # [B, L, D]


        # dec_out = self.dense_feature(dec_out)
        # return dec_out[:, -self.pred_len:,:].squeeze(1)  # [B, L, D]


# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # [B, L, D]
#     model = DLinear_Model(seq_len=32,pred_len=1,enc_in=15,individual=False)
#     out = model(x)
#     print(out.shape)



## TCN

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        """
        其实这就是一个裁剪的模块，裁剪多出来的padding
        """
        # print(x,x.shape)
        # print(x[:, :, :-self.chomp_size].contiguous(),x[:, :, :-self.chomp_size].contiguous().shape)
        # print('以进行裁剪')
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        """
        相当于一个Residual block

        :param n_inputs: int, 输入通道数
        :param n_outputs: int, 输出通道数
        :param kernel_size: int, 卷积核尺寸
        :param stride: int, 步长，一般为1
        :param dilation: int, 膨胀系数
        :param padding: int, 填充系数
        :param dropout: float, dropout比率
        """
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # 经过conv1，输出的size其实是(Batch, input_channel, seq_len + padding)
        self.chomp1 = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        """
        参数初始化
        :return:
        """
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        """
        :param x: size of (Batch, input_channel, seq_len)
        :return:
        """
        # print(self.conv1.weight.shape)
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TCN_Model(nn.Module):
    def __init__(self, num_inputs,seq_len, num_channels,pred_len, kernel_size=2, dropout=0.2,num_experts=2,k=2):
        """
        TCN，目前paper给出的TCN结构很好的支持每个时刻为一个数的情况，即sequence结构，
        对于每个时刻为一个向量这种一维结构，勉强可以把向量拆成若干该时刻的输入通道，
        对于每个时刻为一个矩阵或更高维图像的情况，就不太好办。

        :param num_inputs: int， 输入通道数
        :param num_channels: list，每层的hidden_channel数，例如[25,25,25,25]表示有4个隐层，每层hidden_channel数为25，也就代表有25个卷积核在这次卷积中执行
        :param kernel_size: int, 卷积核尺寸，只需要定义卷积核长度
        :param dropout: float, drop_out比率
        """
        super(TCN_Model, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i  # 膨胀系数：1，2，4，8……
            in_channels = num_inputs if i == 0 else num_channels[i - 1]  # 确定每一层的输入通道数
            out_channels = num_channels[i]  # 确定每一层的输出通道数
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1,
                                     dilation=dilation_size,                        # 膨胀卷积
                                     padding=(kernel_size - 1) * dilation_size,     # 因果卷积
                                     dropout=dropout)]
        self.network = nn.Sequential(*layers)
        self.dense_pred = nn.Linear(seq_len,pred_len)
        self.dense_feature = nn.Linear(num_channels[-1],1)
        
        # self.dense_feature = nn.Linear(num_channels[-1],80)
    
        self.embedding = Channel_Embedding_ablation(num_inputs, out_channels=num_inputs, num_experts=num_experts, k=k)

    def forward(self, x):
        """
        输入x的结构不同于RNN，一般RNN的size为(Batch, seq_len, channels)或者(seq_len, Batch, channels)，
        这里把seq_len放在channels后面，把所有时间步的数据拼起来，当做Conv1d的输入尺寸，实现卷积跨时间步的操作，
        很巧妙的设计。

        为什么输入和输出数据结构是这样的
        :param x: size of (Batch, input_channel, seq_len)
        :return: size of (Batch, output_channel, seq_len)
        """
        B, _, _ = x.shape
        x = x.permute(0, 2, 1)
        x = self.embedding(x)
        x = x.transpose(1, 2)  # B,M,L,D

        
        x = x.permute(0,2,1)
        x = self.network(x)
        x = self.dense_feature(x.permute(0,2,1)).squeeze(-1)
        x = self.dense_pred(x)

        # x = self.dense_feature(x.permute(0,2,1)).permute(0,2,1)
        # x = self.dense_pred(x).squeeze(-1)
        return x



# if __name__ == '__main__':
#     x = torch.randn((128,50,15))        # [B, L, D]
#     model = TemporalConvNet(num_inputs=15, num_channels=[10,5], kernel_size=2,seq_len=50,pred_len=1)
#     out = model(x)
#     print(out.shape)



## ModernTCN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange


class Embedding(nn.Module):
    def __init__(self, P=8, S=4, D=2048):
        super(Embedding, self).__init__()
        self.P = P
        self.S = S
        self.conv = nn.Conv1d(
            in_channels=1,
            out_channels=D,
            kernel_size=P,
            stride=S
        )

    def forward(self, x):
        # x: [B, M, L]
        B = x.shape[0]
        x = x.unsqueeze(2)  # 新增一个维度作为通道数 [B, M, L] -> [B, M, 1, L]
        x = rearrange(x, 'b m r l -> (b m) r l')  # [B, M, 1, L] -> [B*M, 1, L]
        x_pad = F.pad(
            x,
            pad=(0, self.P - self.S),       # 第一个维度不填充，填充第二个维度
            mode='replicate'                # 复制边缘元素的值进行填充
        )  # [B*M, 1, L] -> [B*M, 1, L+P-S]

        x_emb = self.conv(x_pad)  # [B*M, 1, L+P-S] -> [B*M, D, N]
        x_emb = rearrange(x_emb, '(b m) d n -> b m d n', b=B)  # [B*M, D, N] -> [B, M, D, N]

        return x_emb  # x_emb: [B, M, D, N]


class ConvFFN(nn.Module):
    def __init__(self, M, D, r, one=True):  # one is True: ConvFFN1, one is False: ConvFFN2
        super(ConvFFN, self).__init__()
        groups_num = M if one else D
        self.pw_con1 = nn.Conv1d(
            in_channels=M * D,
            out_channels=r * M * D,
            kernel_size=1,
            groups=groups_num
        )
        self.pw_con2 = nn.Conv1d(
            in_channels=r * M * D,
            out_channels=M * D,
            kernel_size=1,
            groups=groups_num
        )

    def forward(self, x):
        # x: [B, M*D, N]
        x = self.pw_con2(F.gelu(self.pw_con1(x)))
        return x  # x: [B, M*D, N]


class ModernTCNBlock(nn.Module):
    def __init__(self, M, D, kernel_size, r):
        super(ModernTCNBlock, self).__init__()
        # 深度分离卷积负责捕获时域关系(输出的通道数并没有改变)
        # 深度卷积的分组数等于输入的通道数，且输入通道等于输出通道
        self.dw_conv = nn.Conv1d(
            in_channels=M * D,
            out_channels=M * D,
            kernel_size=kernel_size,
            groups=M * D,
            padding='same'
        )
        self.bn = nn.BatchNorm1d(M * D)
        self.conv_ffn1 = ConvFFN(M, D, r, one=True)    # 分为M个组
        self.conv_ffn2 = ConvFFN(M, D, r, one=False)   # 分为D个组

    def forward(self, x_emb):
        # x_emb: [B, M, D, N]
        D = x_emb.shape[-2]
        x = rearrange(x_emb, 'b m d n -> b (m d) n')  # [B, M, D, N] -> [B, M*D, N]
        x = self.dw_conv(x)  # [B, M*D, N] -> [B, M*D, N]
        x = self.bn(x)  # [B, M*D, N] -> [B, M*D, N]
        x = self.conv_ffn1(x)  # [B, M*D, N] -> [B, M*D, N]

        x = rearrange(x, 'b (m d) n -> b m d n', d=D)  # [B, M*D, N] -> [B, M, D, N]
        x = x.permute(0, 2, 1, 3)  # [B, M, D, N] -> [B, D, M, N]
        x = rearrange(x, 'b d m n -> b (d m) n')  # [B, D, M, N] -> [B, D*M, N]

        x = self.conv_ffn2(x)  # [B, D*M, N] -> [B, D*M, N]

        x = rearrange(x, 'b (d m) n -> b d m n', d=D)  # [B, D*M, N] -> [B, D, M, N]
        x = x.permute(0, 2, 1, 3)  # [B, D, M, N] -> [B, M, D, N]

        out = x + x_emb
        return out  # out: [B, M, D, N]


class ModernTCN(nn.Module):
    def __init__(self, M, L, T, D=2048, P=8, S=4, kernel_size=51, r=1, num_layers=2,num_experts=2,k=2):
        super(ModernTCN, self).__init__()
        # 深度分离卷积负责捕获时域关系
        self.num_layers = num_layers
        N = L // S
        self.embed_layer = Embedding(P, S, D)
        self.backbone = nn.ModuleList([ModernTCNBlock(M, D, kernel_size, r) for _ in range(num_layers)])
        self.head = nn.Linear(D*N, T)
        self.dense_feature = nn.Linear(M,1)

        # self.dense_feature = nn.Linear(M,80)

        self.embedding = Channel_Embedding_ablation(M, out_channels=M, num_experts=num_experts, k=k)


    def forward(self, x):
        # B, _, _ = x.shape
        # x = x.permute(0, 2, 1)
        # x = self.embedding(x)
        # x = x.transpose(1, 2)  # B,M,L,D


        x = x.permute(0,2,1)        # [B, L, M] -> [B, M, L]
        x_emb = self.embed_layer(x)  # [B, M, L] -> [B, M, D, N]
        # print(x_emb.shape)

        for i in range(self.num_layers):
            x_emb = self.backbone[i](x_emb)  # [B, M, D, N] -> [B, M, D, N]

        # Flatten
        z = rearrange(x_emb, 'b m d n -> b m (d n)')  # [B, M, D, N] -> [B, M, D*N]
        pred = self.dense_feature(z.permute(0,2,1))
        pred = self.head(pred.squeeze(-1))

        # pred = self.dense_feature(z.permute(0,2,1)).permute(0,2,1)
        # pred = self.head(pred).squeeze(-1)
        return pred  # out: [B, M, T]


# B：batch size
# M：多变量序列的变量数
# L：过去序列的长度(时间窗口大小)
# T: 预测序列的长度
# N: 分Patch后Patch的个数
# D：每个变量的通道数（可能是代表着一个变量有多个特征，而每个特征就是一个通道），那么M个变量，也就有M*D个通道，也就是DWConv中的通道数
# P：kernel size of embedding layer
# S：stride of embedding layer


# past_series = torch.rand(2, 4, 96)      # B, M, L
# model = ModernTCN(4, 96, 192)           # 利用时间窗口96预测192，长时间预测
# pred_series = model(past_series)
# print(pred_series.shape)   # torch.Size([2, 4, 192])




# if __name__ == "__main__":
#     x = torch.randn((128,32,15))        # [B, L, D]
#     model = ModernTCN(M=15,L=32,T=1,D=64)
#     out = model(x)
#     print(out.shape)


## Att-CNN-LSTM

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Att_CNN_LSTM(nn.Module):
    def __init__(self, in_channels, out_channels,hidden_size,seq_len,kernel_size,dilation_size,pred_len,num_experts=2,k=2):
        super(Att_CNN_LSTM, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels,kernel_size=kernel_size,
                                           stride=1, padding=(kernel_size - 1) * dilation_size, dilation=dilation_size)
        self.conv = nn.Sequential(
            # nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=3),
            self.conv1,
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1)
        )
        self.dense1 = nn.Linear(seq_len, seq_len)
        self.lstm = nn.LSTM(input_size=out_channels+in_channels, hidden_size=hidden_size,
                            num_layers=2, batch_first=True)
        self.dense2 = nn.Linear(hidden_size, hidden_size)

        self.attention = nn.MultiheadAttention(embed_dim=hidden_size,num_heads=4,batch_first=True,
                                               dropout=0.2)

        self.fc = nn.Linear(hidden_size, 1)
        # self.fc = nn.Linear(hidden_size, 80)

        self.fc_time = nn.Linear(seq_len,pred_len)
        self.dropout = nn.Dropout(0.1)


        self.embedding = Channel_Embedding_ablation(in_channels, out_channels=in_channels, num_experts=num_experts, k=k)


    def forward(self, emb_x):
        '''
        x:(batch,in_channel,seq_len)        作为conv1d输入
          (batch,seq_len-kernel_size+1,out_channel)     # 作为LSTM的输入
        '''
        B, _, _ = emb_x.shape
        x = emb_x.permute(0, 2, 1)
        x = self.embedding(x)
        emb_x = x.transpose(1, 2)  # B,M,L,D



        
        emb_x = emb_x.permute(0,2,1)
        conv_x = self.conv(emb_x)            # (batch,out_channel,seq_len-kernel_size*2+2)       # 因为有池化层的kernel_size
        conv_x = self.dense1(conv_x)
        x = torch.cat((conv_x,emb_x),dim=1)
        x = x.permute(0, 2, 1)      # (batch,seq_len-kernel_size*2+2,out_channel)
        x, _ = self.lstm(x)         # (batch,seq_len-kernel_size*2+2,hidden_size)
        x = self.dropout(x)
        x = self.dense2(x)
        x, _ = self.attention(x,x,x)
        x = self.fc(x).squeeze(-1)
        x = self.fc_time(x)

        # x = self.fc(x).permute(0,2,1)
        # x = self.fc_time(x).squeeze(-1)   
        return x


# if __name__ == '__main__':
#     x = torch.randn((128,50,15))        # [B, L, D]
#     model = Att_CNN_LSTM(in_channels=15,out_channels=16,hidden_size=64,seq_len=50,kernel_size=2,dilation_size=1,pred_len=1)
#     out = model(x)
#     print(out.shape)


## ESN

In [1]:
"""
This examples is not intended to be optimized. Its purpose is to show how to handle
big datasets with multiple sequences. The accuracy should be around 10%.
"""

import re
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.utils.rnn import PackedSequence
import torch.sparse


def apply_permutation(tensor, permutation, dim=1):
    # type: (Tensor, Tensor, int) -> Tensor
    return tensor.index_select(dim, permutation)


class Reservoir(nn.Module):

    def __init__(self, mode, input_size, hidden_size, num_layers, leaking_rate,
                 spectral_radius, w_ih_scale,
                 density, bias=True, batch_first=False):
        super(Reservoir, self).__init__()
        self.mode = mode
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.leaking_rate = leaking_rate
        self.spectral_radius = spectral_radius
        self.w_ih_scale = w_ih_scale
        self.density = density
        self.bias = bias
        self.batch_first = batch_first

        self._all_weights = []
        for layer in range(num_layers):
            layer_input_size = input_size if layer == 0 else hidden_size

            w_ih = nn.Parameter(torch.Tensor(hidden_size, layer_input_size))
            w_hh = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
            b_ih = nn.Parameter(torch.Tensor(hidden_size))
            layer_params = (w_ih, w_hh, b_ih)

            param_names = ['weight_ih_l{}{}', 'weight_hh_l{}{}']
            if bias:
                param_names += ['bias_ih_l{}{}']
            param_names = [x.format(layer, '') for x in param_names]

            for name, param in zip(param_names, layer_params):
                setattr(self, name, param)
            self._all_weights.append(param_names)

        self.reset_parameters()

    def _apply(self, fn):
        ret = super(Reservoir, self)._apply(fn)
        return ret

    def reset_parameters(self):
        weight_dict = self.state_dict()
        for key, value in weight_dict.items():
            if key == 'weight_ih_l0':
                nn.init.uniform_(value, -1, 1)
                value *= self.w_ih_scale[1:]
            elif re.fullmatch('weight_ih_l[^0]*', key):
                nn.init.uniform_(value, -1, 1)
            elif re.fullmatch('bias_ih_l[0-9]*', key):
                nn.init.uniform_(value, -1, 1)
                value *= self.w_ih_scale[0]
            elif re.fullmatch('weight_hh_l[0-9]*', key):
                w_hh = torch.Tensor(self.hidden_size * self.hidden_size)
                w_hh.uniform_(-1, 1)
                if self.density < 1:
                    zero_weights = torch.randperm(
                        int(self.hidden_size * self.hidden_size))
                    zero_weights = zero_weights[
                                   :int(
                                       self.hidden_size * self.hidden_size * (
                                                   1 - self.density))]
                    w_hh[zero_weights] = 0
                w_hh = w_hh.view(self.hidden_size, self.hidden_size)
                abs_eigs = torch.abs(torch.linalg.eigvals(w_hh))
                weight_dict[key] = w_hh * (self.spectral_radius / torch.max(abs_eigs))

        self.load_state_dict(weight_dict)

    def check_input(self, input, batch_sizes):
        # type: (Tensor, Optional[Tensor]) -> None
        expected_input_dim = 2 if batch_sizes is not None else 3
        if input.dim() != expected_input_dim:
            raise RuntimeError(
                'input must have {} dimensions, got {}'.format(
                    expected_input_dim, input.dim()))
        if self.input_size != input.size(-1):
            raise RuntimeError(
                'input.size(-1) must be equal to input_size. Expected {}, got {}'.format(
                    self.input_size, input.size(-1)))

    def get_expected_hidden_size(self, input, batch_sizes):
        # type: (Tensor, Optional[Tensor]) -> Tuple[int, int, int]
        if batch_sizes is not None:
            mini_batch = batch_sizes[0]
            mini_batch = int(mini_batch)
        else:
            mini_batch = input.size(0) if self.batch_first else input.size(1)
        expected_hidden_size = (self.num_layers, mini_batch, self.hidden_size)
        return expected_hidden_size

    def check_hidden_size(self, hx, expected_hidden_size, msg='Expected hidden size {}, got {}'):
        # type: (Tensor, Tuple[int, int, int], str) -> None
        if hx.size() != expected_hidden_size:
            raise RuntimeError(msg.format(expected_hidden_size, tuple(hx.size())))

    def check_forward_args(self, input, hidden, batch_sizes):
        # type: (Tensor, Tensor, Optional[Tensor]) -> None
        self.check_input(input, batch_sizes)
        expected_hidden_size = self.get_expected_hidden_size(input, batch_sizes)

        self.check_hidden_size(hidden, expected_hidden_size)

    def permute_hidden(self, hx, permutation):
        # type: (Tensor, Optional[Tensor]) -> Tensor
        if permutation is None:
            return hx
        return apply_permutation(hx, permutation)

    def forward(self, input, hx=None):
        is_packed = isinstance(input, PackedSequence)
        if is_packed:
            input, batch_sizes, sorted_indices, unsorted_indices = input
            max_batch_size = int(batch_sizes[0])
        else:
            batch_sizes = None
            max_batch_size = input.size(0) if self.batch_first else input.size(1)
            sorted_indices = None
            unsorted_indices = None

        if hx is None:
            hx = input.new_zeros(self.num_layers, max_batch_size,
                                 self.hidden_size, requires_grad=False)
        else:
            # Each batch of the hidden state should match the input sequence that
            # the user believes he/she is passing in.
            hx = self.permute_hidden(hx, sorted_indices)

        flat_weight = None

        self.check_forward_args(input, hx, batch_sizes)
        func = AutogradReservoir(
            self.mode,
            self.input_size,
            self.hidden_size,
            num_layers=self.num_layers,
            batch_first=self.batch_first,
            train=self.training,
            variable_length=is_packed,
            flat_weight=flat_weight,
            leaking_rate=self.leaking_rate
        )
        output, hidden = func(input, self.all_weights, hx, batch_sizes)
        if is_packed:
            output = PackedSequence(output, batch_sizes, sorted_indices, unsorted_indices)
        return output, self.permute_hidden(hidden, unsorted_indices)

    def extra_repr(self):
        s = '({input_size}, {hidden_size}'
        if self.num_layers != 1:
            s += ', num_layers={num_layers}'
        if self.bias is not True:
            s += ', bias={bias}'
        if self.batch_first is not False:
            s += ', batch_first={batch_first}'
        s += ')'
        return s.format(**self.__dict__)

    def __setstate__(self, d):
        super(Reservoir, self).__setstate__(d)
        self.__dict__.setdefault('_data_ptrs', [])
        if 'all_weights' in d:
            self._all_weights = d['all_weights']
        if isinstance(self._all_weights[0][0], str):
            return
        num_layers = self.num_layers
        self._all_weights = []
        for layer in range(num_layers):
            weights = ['weight_ih_l{}{}', 'weight_hh_l{}{}', 'bias_ih_l{}{}']
            weights = [x.format(layer) for x in weights]
            if self.bias:
                self._all_weights += [weights]
            else:
                self._all_weights += [weights[:2]]

    @property
    def all_weights(self):
        return [[getattr(self, weight) for weight in weights] for weights in
                self._all_weights]


def AutogradReservoir(mode, input_size, hidden_size, num_layers=1,
                      batch_first=False, train=True,
                      batch_sizes=None, variable_length=False, flat_weight=None,
                      leaking_rate=1):
    if mode == 'RES_TANH':
        cell = ResTanhCell
    elif mode == 'RES_RELU':
        cell = ResReLUCell
    elif mode == 'RES_ID':
        cell = ResIdCell

    if variable_length:
        layer = (VariableRecurrent(cell, leaking_rate),)
    else:
        layer = (Recurrent(cell, leaking_rate),)

    func = StackedRNN(layer,
                      num_layers,
                      False,
                      train=train)

    def forward(input, weight, hidden, batch_sizes):
        if batch_first and batch_sizes is None:
            input = input.transpose(0, 1)

        nexth, output = func(input, hidden, weight, batch_sizes)

        if batch_first and not variable_length:
            output = output.transpose(0, 1)

        return output, nexth

    return forward


def Recurrent(inner, leaking_rate):
    def forward(input, hidden, weight, batch_sizes):
        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden = inner(input[i], hidden, leaking_rate, *weight)
            # hack to handle LSTM
            output.append(hidden[0] if isinstance(hidden, tuple) else hidden)

        output = torch.cat(output, 0).view(input.size(0), *output[0].size())

        return hidden, output

    return forward


def VariableRecurrent(inner, leaking_rate):
    def forward(input, hidden, weight, batch_sizes):
        output = []
        input_offset = 0
        last_batch_size = batch_sizes[0]
        hiddens = []
        flat_hidden = not isinstance(hidden, tuple)
        if flat_hidden:
            hidden = (hidden,)
        for batch_size in batch_sizes:
            step_input = input[input_offset:input_offset + batch_size]
            input_offset += batch_size

            dec = last_batch_size - batch_size
            if dec > 0:
                hiddens.append(tuple(h[-dec:] for h in hidden))
                hidden = tuple(h[:-dec] for h in hidden)
            last_batch_size = batch_size

            if flat_hidden:
                hidden = (inner(step_input, hidden[0], leaking_rate, *weight),)
            else:
                hidden = inner(step_input, hidden, leaking_rate, *weight)

            output.append(hidden[0])
        hiddens.append(hidden)
        hiddens.reverse()

        hidden = tuple(torch.cat(h, 0) for h in zip(*hiddens))
        assert hidden[0].size(0) == batch_sizes[0]
        if flat_hidden:
            hidden = hidden[0]
        output = torch.cat(output, 0)

        return hidden, output

    return forward


def StackedRNN(inners, num_layers, lstm=False, train=True):
    num_directions = len(inners)
    total_layers = num_layers * num_directions

    def forward(input, hidden, weight, batch_sizes):
        assert (len(weight) == total_layers)
        next_hidden = []
        all_layers_output = []

        for i in range(num_layers):
            all_output = []
            for j, inner in enumerate(inners):
                l = i * num_directions + j

                hy, output = inner(input, hidden[l], weight[l], batch_sizes)
                next_hidden.append(hy)
                all_output.append(output)

            input = torch.cat(all_output, input.dim() - 1)
            all_layers_output.append(input)

        all_layers_output = torch.cat(all_layers_output, -1)
        next_hidden = torch.cat(next_hidden, 0).view(
            total_layers, *next_hidden[0].size())

        return next_hidden, all_layers_output

    return forward


def ResTanhCell(input, hidden, leaking_rate, w_ih, w_hh, b_ih=None):
    hy_ = torch.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh))
    hy = (1 - leaking_rate) * hidden + leaking_rate * hy_
    return hy


def ResReLUCell(input, hidden, leaking_rate, w_ih, w_hh, b_ih=None):
    hy_ = F.relu(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh))
    hy = (1 - leaking_rate) * hidden + leaking_rate * hy_
    return hy


def ResIdCell(input, hidden, leaking_rate, w_ih, w_hh, b_ih=None):
    hy_ = F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh)
    hy = (1 - leaking_rate) * hidden + leaking_rate * hy_
    return hy

In [23]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import PackedSequence, pad_packed_sequence


def prepare_target(target, seq_lengths, washout, batch_first=False):
    """ Preprocess target for offline training.

    Args:
        target (seq_len, batch, output_size): tensor containing
            the features of the target sequence.
        seq_lengths: list of lengths of each sequence in the batch.
        washout: number of initial timesteps during which output of the
            reservoir is not forwarded to the readout. One value per sample.
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False``

    Returns:
        tensor containing the features of the batch's sequences rolled out along
        one axis, minus the washouts and the padded values.
    """

    if batch_first:
        target = target.transpose(0, 1)
    n_sequences = target.size(1)
    target_dim = target.size(2)
    train_len = sum(torch.tensor(seq_lengths) - torch.tensor(washout)).item()

    new_target = torch.zeros(train_len, target_dim, device=target.device)

    idx = 0
    for s in range(n_sequences):
        batch_len = seq_lengths[s] - washout[s]
        new_target[idx:idx + batch_len, :] = target[washout[s]:seq_lengths[s], s, :]
        idx += batch_len

    return new_target


def washout_tensor(tensor, washout, seq_lengths, bidirectional=False, batch_first=False):
    tensor = tensor.transpose(0, 1) if batch_first else tensor.clone()
    if type(seq_lengths) == list:
        seq_lengths = seq_lengths.copy()
    if type(seq_lengths) == torch.Tensor:
        seq_lengths = seq_lengths.clone()

    for b in range(tensor.size(1)):
        if washout[b] > 0:
            tmp = tensor[washout[b]:seq_lengths[b], b].clone()
            tensor[:seq_lengths[b] - washout[b], b] = tmp
            tensor[seq_lengths[b] - washout[b]:, b] = 0
            seq_lengths[b] -= washout[b]

            if bidirectional:
                tensor[seq_lengths[b] - washout[b]:, b] = 0
                seq_lengths[b] -= washout[b]

    if type(seq_lengths) == list:
        max_len = max(seq_lengths)
    else:
        max_len = max(seq_lengths).item()

    return tensor[:max_len], seq_lengths


class ESN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,time_len=32, pred_len=1, num_layers=1,
                 nonlinearity='tanh', batch_first=False, leaking_rate=1,
                 spectral_radius=0.9, w_ih_scale=1, lambda_reg=0, density=1,
                 w_io=False, readout_training='svd', output_steps='all'
                ,num_experts=3,k=3
                ):
        super(ESN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        if nonlinearity == 'tanh':
            mode = 'RES_TANH'
        elif nonlinearity == 'relu':
            mode = 'RES_RELU'
        elif nonlinearity == 'id':
            mode = 'RES_ID'
        else:
            raise ValueError("Unknown nonlinearity '{}'".format(nonlinearity))
        self.batch_first = batch_first
        self.leaking_rate = leaking_rate
        self.spectral_radius = spectral_radius
        if type(w_ih_scale) != torch.Tensor:
            self.w_ih_scale = torch.ones(input_size + 1)
            self.w_ih_scale *= w_ih_scale
        else:
            self.w_ih_scale = w_ih_scale

        self.lambda_reg = lambda_reg
        self.density = density
        self.w_io = w_io
        if readout_training in {'gd', 'svd', 'cholesky', 'inv'}:
            self.readout_training = readout_training
        else:
            raise ValueError("Unknown readout training algorithm '{}'".format(
                readout_training))

        self.reservoir = Reservoir(mode, input_size, hidden_size, num_layers,
                                   leaking_rate, spectral_radius,
                                   self.w_ih_scale, density,
                                   batch_first=batch_first)

        if w_io:
            self.readout = nn.Linear(input_size + hidden_size * num_layers,
                                     output_size)
        else:
            self.readout = nn.Linear(hidden_size * num_layers, output_size)
        if readout_training == 'offline':
            self.readout.weight.requires_grad = False

        if output_steps in {'all', 'mean', 'last'}:
            self.output_steps = output_steps
        else:
            raise ValueError("Unknown task '{}'".format(
                output_steps))

        self.XTX = None
        self.XTy = None
        self.X = None

        self.dense_time = nn.Linear(time_len-int(0.2 * time_len),pred_len)


        self.embedding = Channel_Embedding_ablation(input_size, out_channels=input_size, num_experts=num_experts, k=k)


    def forward(self, input, h_0=None, target=None):
        B, _, _ = input.shape
        x = input.permute(0, 2, 1)
        x = self.embedding(x)
        input = x.transpose(1, 2)  # B,M,L,D
        
        input = input.permute(1,0,2)
        washout = [int(0.2 * input.size(0))] * input.size(1)

        with torch.no_grad():
            is_packed = isinstance(input, PackedSequence)

            output, hidden = self.reservoir(input, h_0)
            if is_packed:
                output, seq_lengths = pad_packed_sequence(output, batch_first=self.batch_first)
            else:
                if self.batch_first:
                    seq_lengths = output.size(0) * [output.size(1)]
                else:
                    seq_lengths = output.size(1) * [output.size(0)]

            if self.batch_first:
                output = output.transpose(0, 1)

            output, seq_lengths = washout_tensor(output, washout, seq_lengths)

            if self.w_io:
                if is_packed:
                    input, input_lengths = pad_packed_sequence(input,
                                                          batch_first=self.batch_first)
                else:
                    input_lengths = [input.size(0)] * input.size(1)

                if self.batch_first:
                    input = input.transpose(0, 1)

                input, _ = washout_tensor(input, washout, input_lengths)
                output = torch.cat([input, output], -1)

            if self.readout_training == 'gd' or target is None:
                with torch.enable_grad():
                    output = self.readout(output)

                    if is_packed:
                        for i in range(output.size(1)):
                            if seq_lengths[i] < output.size(0):
                                output[seq_lengths[i]:, i] = 0

                    if self.batch_first:
                        output = output.transpose(0, 1)

                    # Uncomment if you want packed output.
                    if is_packed:
                        output = pack_padded_sequence(output, seq_lengths,
                                                      batch_first=self.batch_first)
                    # print(output.shape)
                    output = self.dense_time(output.squeeze(-1).permute(1,0))

                    # output = self.dense_time(output.permute(1,2,0)).squeeze(-1)
                    return output

            else:
                batch_size = output.size(1)

                X = torch.ones(target.size(0), 1 + output.size(2), device=target.device)
                row = 0
                for s in range(batch_size):
                    if self.output_steps == 'all':
                        X[row:row + seq_lengths[s], 1:] = output[:seq_lengths[s],
                                                          s]
                        row += seq_lengths[s]
                    elif self.output_steps == 'mean':
                        X[row, 1:] = torch.mean(output[:seq_lengths[s], s], 0)
                        row += 1
                    elif self.output_steps == 'last':
                        X[row, 1:] = output[seq_lengths[s] - 1, s]
                        row += 1

                if self.readout_training == 'cholesky':
                    if self.XTX is None:
                        self.XTX = torch.mm(X.t(), X)
                        self.XTy = torch.mm(X.t(), target)
                    else:
                        self.XTX += torch.mm(X.t(), X)
                        self.XTy += torch.mm(X.t(), target)

                elif self.readout_training == 'svd':
                    # Scikit-Learn SVD solver for ridge regression.
                    U, s, V = torch.svd(X)
                    idx = s > 1e-15  # same default value as scipy.linalg.pinv
                    s_nnz = s[idx][:, None]
                    UTy = torch.mm(U.t(), target)
                    d = torch.zeros(s.size(0), 1, device=X.device)
                    d[idx] = s_nnz / (s_nnz ** 2 + self.lambda_reg)
                    d_UT_y = d * UTy
                    W = torch.mm(V, d_UT_y).t()

                    self.readout.bias = nn.Parameter(W[:, 0])
                    self.readout.weight = nn.Parameter(W[:, 1:])
                elif self.readout_training == 'inv':
                    self.X = X
                    if self.XTX is None:
                        self.XTX = torch.mm(X.t(), X)
                        self.XTy = torch.mm(X.t(), target)
                    else:
                        self.XTX += torch.mm(X.t(), X)
                        self.XTy += torch.mm(X.t(), target)
                return None

    def fit(self):
        if self.readout_training in {'gd', 'svd'}:
            return

        if self.readout_training == 'cholesky':
            W = torch.linalg.solve(self.XTy,
                                   self.XTX + self.lambda_reg * torch.eye(
                                       self.XTX.size(0), device=self.XTX.device))[0].t()
            self.XTX = None
            self.XTy = None

            self.readout.bias = nn.Parameter(W[:, 0])
            self.readout.weight = nn.Parameter(W[:, 1:])
        elif self.readout_training == 'inv':
            I = (self.lambda_reg * torch.eye(self.XTX.size(0))).to(
                self.XTX.device)
            A = self.XTX + I
            X_rank = torch.linalg.matrix_rank(A).item()

            if X_rank == self.X.size(0):
                W = torch.mm(torch.inverse(A), self.XTy).t()
            else:
                W = torch.mm(torch.pinverse(A), self.XTy).t()

            self.readout.bias = nn.Parameter(W[:, 0])
            self.readout.weight = nn.Parameter(W[:, 1:])

            self.XTX = None
            self.XTy = None

    def reset_parameters(self):
        self.reservoir.reset_parameters()
        self.readout.reset_parameters()


# if __name__ == '__main__':
#     x = torch.randn((128,32,15))
#     washout_list = [int(0.2 * x.size(1))] * x.size(0)

#     model = ESN(input_size=15,hidden_size=30,output_size=1,num_layers=3,pred_len=1)
#     out = model(x)
#     model.fit()
#     print(out.shape)


# 运行模型

In [24]:
import os
import time
import torch
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


# 环境设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # 使用gpu
batch_size = 128
criterion = nn.MSELoss()
start_time = time.time()

# 能固定dataloader的结果，使复现结果一致
def setup_seed(seed):
   torch.manual_seed(seed)
   torch.cuda.manual_seed_all(seed)
   np.random.seed(seed)
   random.seed(seed)
   torch.backends.cudnn.deterministic = True
# setup_seed(20)


def mean_squared_error(y_true, y_pred):
  # 确保输入数据长度一致
    if len(y_true) != len(y_pred):
        raise ValueError("长度不一致：y_true 和 y_pred")

      # 计算MSE
    squared_errors = [(y_true[i] - y_pred[i])**2 for i in range(len(y_true))]
    return sum(squared_errors) / len(y_true)


def train(model, optimizer, scheduler, epoch, train_dataloader, train_seq, type_loss):
    model.train()  # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for batch_index,batch_data in enumerate(train_dataloader):
        data,targets = batch_data       # torch.Size([B, L, D]) torch.Size([B, pred_len])
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        output = model(data)
        if type_loss == 'mse':
            loss = criterion(output, targets)
        else:
            loss, loss_shape, loss_temporal = dilate_loss(output, targets, alpha=0.8, gamma=0.01, device=device)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # 防止梯度爆炸或梯度消失问题
        optimizer.step()
        total_loss += loss.item()
        log_interval = int(len(train_seq) / batch_size/5 )
        if batch_index % log_interval == 0 and batch_index > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            # print('| epoch {:3d} | {:5d}/{:5d} batches | '
            #       'lr {:02.6f} | {:5.2f} ms | '
            #       'loss {:5.5f}'.format(
            #     epoch, batch_index, len(train_seq) // batch_size, scheduler.get_lr()[0],
            #                   elapsed * 1000 / log_interval,cur_loss))  # , math.exp(cur_loss)
            total_loss = 0
            start_time = time.time()
    return model


def evaluate(eval_model, dataloader, data_source_x,type_loss):
    eval_model.eval()  # Turn on the evaluation mode
    total_loss = 0.
    result = torch.Tensor(0).to(device)
    truth = torch.Tensor(0).to(device)
    with torch.no_grad():
        for batch_index,dataset in enumerate(dataloader):
            data, targets = dataset
            data = data.to(device)
            targets = targets.to(device)
            output = eval_model(data)
            if type_loss == 'mse':
                loss = criterion(output, targets)
                total_loss +=  len(data)*loss.item()
            else:
                loss, loss_shape, loss_temporal = dilate_loss(output, targets, alpha=0.8, gamma=0.01, device=device)
                total_loss +=  len(data)*loss.item()
            result = torch.cat((result.cpu(), output.cpu()),0)  # todo: check this. -> looks good to me
            truth = torch.cat((truth.cpu(), targets.cpu()), 0)
    return total_loss / len(data_source_x),result,truth


def get_model(type_name, model_name, load_name, pred_len=1,time_len=32):
    model = None        # 定义模型

    if load_name == 'sr':
        if model_name == 'LSTM':
            if type_name == 'lorenz':
                # model = LSTM_Model(seq_len=32, input_dim=15, hidden_dim1=128, hidden_dim2=32, pred_len=pred_len)
                model = LSTM_Model(seq_len=30, input_dim=15, hidden_dim1=128, hidden_dim2=32, pred_len=pred_len,num_experts=4,k=2)

            if type_name == 'power':
                # model = LSTM_Model(seq_len=32, input_dim=30, hidden_dim1=128, hidden_dim2=64, pred_len=pred_len)
                model = LSTM_Model(seq_len=30, input_dim=30, hidden_dim1=128, hidden_dim2=64, pred_len=pred_len,num_experts=8,k=2)

        if model_name == 'Transformer':
            if type_name == 'lorenz':
                model = Transformer_Model(seq_len=30, pred_len=pred_len, enc_in=15,num_experts=4,k=2)
            if type_name == 'power':
                model = Transformer_Model(seq_len=30, pred_len=pred_len, enc_in=30,num_experts=8,k=2)
                
        if model_name == 'DLinear':
            if type_name == 'lorenz':
                model = DLinear_Model(seq_len=30,pred_len=pred_len,enc_in=15,individual=False,num_experts=4,k=2)
            if type_name == 'power':
                model = DLinear_Model(seq_len=30,pred_len=pred_len,enc_in=30,individual=False,num_experts=8,k=2)

        if model_name == 'TCN':
            if type_name == 'lorenz':
                model = TCN_Model(num_inputs=15, num_channels=[10,5], kernel_size=2,seq_len=30,pred_len=pred_len,num_experts=3,k=2)
            if type_name == 'power':
                model = TCN_Model(num_inputs=30, num_channels=[10,5], kernel_size=2,seq_len=30,pred_len=pred_len,num_experts=8,k=2)

        if model_name == 'ModernTCN':
            if type_name == 'lorenz':
                model = ModernTCN(M=15,L=30,T=pred_len,D=64,num_experts=3,k=2)
            if type_name == 'power':
                model = ModernTCN(M=30,L=32,T=pred_len,D=32,num_experts=8,k=2)
        
        if model_name == 'Att_CNN_LSTM':
            if type_name == 'lorenz':
                model = Att_CNN_LSTM(in_channels=15,out_channels=16,hidden_size=128,seq_len=30,kernel_size=2,dilation_size=1,pred_len=pred_len,num_experts=3,k=2)
            if type_name == 'power':
                model = Att_CNN_LSTM(in_channels=30,out_channels=16,hidden_size=128,seq_len=30,kernel_size=2,dilation_size=1,pred_len=pred_len,num_experts=8,k=2)
        
        if model_name == 'ESN':
            if type_name == 'lorenz':
                model = ESN(input_size=15, hidden_size=200,time_len=30, output_size=1,pred_len=pred_len, num_layers=1,num_experts=2,k=2)
            if type_name == 'power':
                model = ESN(input_size=30, hidden_size=200, time_len=30,output_size=1,pred_len=pred_len, num_layers=1,num_experts=4,k=2)

    else:
        if model_name == 'LSTM':
            if type_name == 'lorenz':
                model = LSTM_Model(seq_len=time_len, input_dim=15, hidden_dim1=128, hidden_dim2=64, pred_len=pred_len)
            if type_name == 'rossler':
                model = LSTM_Model(seq_len=time_len, input_dim=13, hidden_dim1=128, hidden_dim2=64, pred_len=pred_len)
            if type_name == 'power':
                model = LSTM_Model(seq_len=time_len, input_dim=30, hidden_dim1=128, hidden_dim2=64, pred_len=pred_len)
            if type_name == 'lorenz-96':
                model = LSTM_Model(seq_len=time_len, input_dim=40, hidden_dim1=128, hidden_dim2=64, pred_len=pred_len)
            if type_name == 'KS':
                model = LSTM_Model(seq_len=time_len, input_dim=80, hidden_dim1=64, hidden_dim2=32, pred_len=pred_len)

        
        if model_name == 'Transformer':
            if type_name == 'lorenz':
                model = Transformer_Model(seq_len=time_len, pred_len=pred_len, enc_in=15)
            if type_name == 'rossler':
                model = Transformer_Model(seq_len=time_len, pred_len=pred_len, enc_in=13)
            if type_name == 'power':
                model = Transformer_Model(seq_len=time_len, pred_len=pred_len, enc_in=30)
            if type_name == 'lorenz-96':
                model = Transformer_Model(seq_len=time_len, pred_len=pred_len, enc_in=40)
            if type_name == 'KS':
                model = Transformer_Model(seq_len=time_len, pred_len=pred_len, enc_in=80)

        
        if model_name == 'DLinear':
            if type_name == 'lorenz':
                model = DLinear_Model(seq_len=time_len,pred_len=pred_len,enc_in=15,individual=False)
            if type_name == 'rossler':
                model = DLinear_Model(seq_len=time_len,pred_len=pred_len,enc_in=13,individual=False)
            if type_name == 'power':
                model = DLinear_Model(seq_len=time_len,pred_len=pred_len,enc_in=30,individual=False)
            if type_name == 'lorenz-96':
                model = DLinear_Model(seq_len=time_len,pred_len=pred_len,enc_in=40,individual=False)
            if type_name == 'KS':
                model = DLinear_Model(seq_len=time_len,pred_len=pred_len,enc_in=80,individual=False)                
                
        if model_name == 'TCN':
            if type_name == 'lorenz':
                model = TCN_Model(num_inputs=15, num_channels=[10,5], kernel_size=2,seq_len=time_len,pred_len=pred_len)
            if type_name == 'rossler':
                model = TCN_Model(num_inputs=13, num_channels=[10,5], kernel_size=2,seq_len=time_len,pred_len=pred_len)            
            if type_name == 'power':
                model = TCN_Model(num_inputs=30, num_channels=[10,5], kernel_size=2,seq_len=time_len,pred_len=pred_len)
            if type_name == 'lorenz-96':
                model = TCN_Model(num_inputs=40, num_channels=[10,5], kernel_size=2,seq_len=time_len,pred_len=pred_len)
            if type_name == 'KS':
                model = TCN_Model(num_inputs=80, num_channels=[10,5], kernel_size=2,seq_len=time_len,pred_len=pred_len)

        
        if model_name == 'ModernTCN':
            if type_name == 'lorenz':
                model = ModernTCN(M=15,L=time_len,T=pred_len,D=64)
            if type_name == 'rossler':
                model = ModernTCN(M=13,L=time_len,T=pred_len,D=64)            
            if type_name == 'power':
                model = ModernTCN(M=30,L=time_len,T=pred_len,D=64)
            if type_name == 'lorenz-96':
                model = ModernTCN(M=40,L=time_len,T=pred_len,D=64)
            if type_name == 'KS':
                model = ModernTCN(M=80,L=time_len,T=pred_len,D=64)

                
        if model_name == 'Att_CNN_LSTM':
            if type_name == 'lorenz':
                model = Att_CNN_LSTM(in_channels=15,out_channels=16,hidden_size=128,seq_len=time_len,kernel_size=2,dilation_size=1,pred_len=pred_len)
            if type_name == 'rossler':
                model = Att_CNN_LSTM(in_channels=13,out_channels=16,hidden_size=128,seq_len=time_len,kernel_size=2,dilation_size=1,pred_len=pred_len)
            if type_name == 'power':
                model = Att_CNN_LSTM(in_channels=30,out_channels=16,hidden_size=128,seq_len=time_len,kernel_size=2,dilation_size=1,pred_len=pred_len)
            if type_name == 'lorenz-96':
                model = Att_CNN_LSTM(in_channels=40,out_channels=16,hidden_size=128,seq_len=time_len,kernel_size=2,dilation_size=1,pred_len=pred_len)
            if type_name == 'KS':
                model = Att_CNN_LSTM(in_channels=80,out_channels=32,hidden_size=256,seq_len=time_len,kernel_size=2,dilation_size=1,pred_len=pred_len)

                
        if model_name == 'ESN':
            if type_name == 'lorenz':
                model = ESN(input_size=15, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
            if type_name == 'rossler':
                model = ESN(input_size=13, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
            if type_name == 'power':
                model = ESN(input_size=30, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
            if type_name == 'lorenz-96':
                model = ESN(input_size=40, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
            if type_name == 'KS':
                model = ESN(input_size=80, hidden_size=200, output_size=80, num_layers=1,pred_len=pred_len,time_len=time_len)
                
        if model_name == 'WESN':
            if type_name == 'lorenz':
                model = ESN(input_size=18, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
            if type_name == 'rossler':
                model = ESN(input_size=18, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
            if type_name == 'power':
                model = ESN(input_size=36, hidden_size=300, output_size=1, num_layers=1,pred_len=pred_len,time_len=time_len)
                
    return model


def read_npy(type_name,model_name,epochs,load_name,weight_decay,lr = 0.001,seed=0,pred_len=1,time_len=32):
    if pred_len == 1:
        if model_name == 'WESN':
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_Wave_{}_{}.npy'.format(type_name,seed),allow_pickle=True).item()
        else:
            if load_name == 'sr':
                saved_dict = np.load(r'/kaggle/input/original-chaotic-data/sr_{}_{}.npy'.format(type_name,seed),allow_pickle=True).item()
            else:
                saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_{}_{}.npy'.format(type_name,seed),allow_pickle=True).item()
        # saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_KS_42.npy',allow_pickle=True).item()

    else:
        # if model_name == 'WESN':
        #     saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_Wave_{}_42_({}).npy'.format(type_name,pred_len),allow_pickle=True).item()
        # else:
        #     print('looking here!')
        #     # saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_{}_{}_({}).npy'.format(type_name,seed,pred_len),allow_pickle=True).item()
        #     saved_dict = np.load(r'/kaggle/input/original-chaotic-data/sr_{}_({}_{}).npy'.format(type_name,time_len,pred_len),allow_pickle=True).item()
       if model_name == 'WESN':
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/tr_Wave_{}_({}_{}).npy'.format(type_name,time_len,pred_len),allow_pickle=True).item()
       else:
            saved_dict = np.load(r'/kaggle/input/original-chaotic-data/sr_{}_({}_{}).npy'.format(type_name,time_len,pred_len),allow_pickle=True).item()

    
    
    train_seq = saved_dict['train']['train_seq'].to(device)
    train_label = saved_dict['train']['train_label'].to(device)
    train_dataloader = saved_dict['train']['train_dataloader']

    val_seq = saved_dict['val']['val_seq'].to(device)
    val_label = saved_dict['val']['val_label'].to(device)
    val_dataloader = saved_dict['val']['val_dataloader']

    test_seq = saved_dict['test']['test_seq'].to(device)
    test_label = saved_dict['test']['test_label'].to(device)
    test_dataloader = saved_dict['test']['test_dataloader']

    print('----------train--------')
    print(f'train X shape:{train_seq.shape}')
    print(f'train y shape:{train_label.shape}')

    print('----------val--------')
    print(f'val X shape:{val_seq.shape}')
    print(f'val y shape:{val_label.shape}')

    print('----------test--------')
    print(f'test X shape:{test_seq.shape}')
    print(f'test y shape:{test_label.shape}')

    model = get_model(type_name, model_name, load_name,pred_len=pred_len,time_len=time_len).to(device)

    type_loss = 'mse'
    # type_loss = 'dilate'

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.98)       # 学习率调度器对象,调度器会在每个10个epoch后将学习率乘以gamma。
    best_test_loss = 1000
    best_val_output, best_val_target,best_test_output, best_test_target = 0, 0, 0, 0
    best_model = None
    list_train_loss, list_test_loss = [], []

    # 训练模型
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train_model = train(model, optimizer, scheduler, epoch,train_dataloader, train_seq,type_loss)

        train_loss, train_output, train_target = evaluate(train_model, train_dataloader, train_seq,type_loss)
        test_loss, test_output, test_target = evaluate(train_model, test_dataloader, test_seq,type_loss)
        list_train_loss.append(train_loss)
        list_test_loss.append(test_loss)

        if test_loss<best_test_loss:
            best_test_loss = test_loss
            best_test_output = test_output
            best_test_target = test_target
            best_model = train_model

        print('| end of epoch {:3d} | time: {:5.2f}s  | train loss {:.10f}| test loss {:.10f} '.format(
            epoch, (time.time() - epoch_start_time), train_loss, test_loss))
        print('-' * 90)
        
        scheduler.step()        # 学习率调度器对象

    data_out = pd.DataFrame({'true':[i[0] for i in best_test_target.tolist()],
                             'pred':[i[0] for i in best_test_output.tolist()]},
                            index=range(len(best_test_output)))
    
    print(data_out)
    print(mean_squared_error(best_test_target,best_test_output))

    # 指定CSV文件的保存路径和文件名
    folder_path = '/kaggle/working/new1/'

    
    # 检查文件夹是否存在，如果不存在则创建
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    if type_name!='KS':
        if pred_len == 1:
            data_out = pd.DataFrame({'true':[i[0] for i in best_test_target.tolist()],
                                     'pred':[i[0] for i in best_test_output.tolist()]},
                                      index=range(len(best_test_output)))
            file_name = 'MOE_{}_{}_{}_{}.csv'.format(load_name,type_name,model_name,seed)
            file_path = os.path.join(folder_path, file_name)
            data_out.to_csv(file_path)
            
        else:
            columns = ['true{}'.format(i+1) for i in range(pred_len)]
            columns.extend(['pred{}'.format(i+1) for i in range(pred_len)])
            true_df = pd.DataFrame(best_test_target.numpy(), columns=columns[:pred_len])
            pred_df = pd.DataFrame(best_test_output.numpy(), columns=columns[pred_len:])
            data_out = pd.concat([true_df, pred_df], axis=1)
            print(data_out)
            
            # get_all_result(true_df, pred_df)
            # file_name = r'{}_{}_{}_{}_({}).csv'.format(load_name,type_name,model_name,seed,pred_len)
            file_name = r'{}_{}_{}_({}_{}).csv'.format(load_name,type_name,model_name,time_len,pred_len)
    
            file_path = os.path.join(folder_path, file_name)
            data_out.to_csv(file_path)
    else:
        columns = ['true{}'.format(i+1) for i in range(80)]
        columns.extend(['pred{}'.format(i+1) for i in range(80)])
        true_df = pd.DataFrame(best_test_target.numpy(), columns=columns[:80])
        pred_df = pd.DataFrame(best_test_output.numpy(), columns=columns[80:])
        data_out = pd.concat([true_df, pred_df], axis=1)
        print(data_out)
        
        # get_all_result(true_df, pred_df)
        file_name = r'{}_{}_{}.csv'.format(load_name,type_name,model_name,time_len,pred_len)

        file_path = os.path.join(folder_path, file_name)
        data_out.to_csv(file_path)
    

    # plt.plot(best_test_target, label='真实值')
    # plt.plot(best_test_output, label='预测值')
    # plt.legend()
    # plt.show()


# model_name = 'ESN'
# read_npy('lorenz',model_name,epochs = 300,load_name='sr', weight_decay=0.02,lr = 0.001)
# read_npy('power',model_name, epochs = 300,load_name='sr', weight_decay=0.02,lr = 0.001)


# for time_len in [24,32,40,48,56,64]:
# for time_len in [8,16]:
#     for model_name in ['WESN']:
#         read_npy('lorenz',model_name, epochs = 300, load_name='tr', weight_decay=0.02,lr = 0.001,seed=42,pred_len=6,time_len=time_len)
#         read_npy('power',model_name, epochs = 300, load_name='tr', weight_decay=0.5,lr = 0.001,seed=42,pred_len=6,time_len=time_len)



# for i in [2,3,4,5,6,7,8,9,10,11,12]:
#     for model_name in ['ESN']:
#         print(i)
#         # read_npy('lorenz',model_name, epochs = 300, load_name='tr', weight_decay=0.02,lr = 0.001,seed=42,pred_len=i)
#         # read_npy('rossler',model_name, epochs = 300, load_name='tr', weight_decay=0.02,lr = 0.001,seed=42,pred_len=i)
#         read_npy('power',model_name, epochs = 300, load_name='tr', weight_decay=0.5,lr = 0.001,seed=42,pred_len=i)

    
    # model_name = 'Transformer'
    # read_npy('power',model_name, epochs = 300, load_name='tr', weight_decay=0.5,lr = 0.001,seed=42,pred_len=i)

    # model_name = 'DLinear'
    # read_npy('power',model_name, epochs = 300, load_name='tr', weight_decay=0.5,lr = 0.001,seed=42,pred_len=i)


for i in [1,2,3,4,5]:
    for model_name in ['ESN']:
        print(i, model_name)
        # model_name = 'LSTM'
        read_npy('lorenz',model_name, epochs = 300, load_name='sr', weight_decay=0.02,lr = 0.001,seed=i)
        # read_npy('power',model_name,epochs = 300,load_name='sr', weight_decay=0.5,lr = 0.001,seed=i)


# for model_name in ['Att_CNN_LSTM']:
#     read_npy('lorenz-96',model_name, epochs = 300, load_name='tr', weight_decay=0.9,lr = 0.001,seed=1,pred_len=1)


#     read_npy('lorenz',model_name, epochs = 300, load_name='tr', weight_decay=0.02,lr = 0.001,seed=i)
#     read_npy('lorenz',model_name, epochs = 300, load_name='sr', weight_decay=0.02,lr = 0.001,seed=i)

    # read_npy('rossler',model_name, epochs = 300, load_name='tr', weight_decay=0.02,lr = 0.001,seed=i)

    # read_npy('power',model_name,epochs = 300,load_name='tr', weight_decay=0.5,lr = 0.001,seed=i)
    # read_npy('power',model_name,epochs = 300,load_name='sr', weight_decay=0.5,lr = 0.001,seed=i)


1 ESN
----------train--------
train X shape:torch.Size([2550, 32, 15])
train y shape:torch.Size([2550, 1])
----------val--------
val X shape:torch.Size([450, 32, 15])
val y shape:torch.Size([450, 1])
----------test--------
test X shape:torch.Size([936, 32, 15])
test y shape:torch.Size([936, 1])
| end of epoch   1 | time:  0.56s  | train loss 0.1537946478| test loss 0.1413835260 
------------------------------------------------------------------------------------------
| end of epoch   2 | time:  0.57s  | train loss 0.0798582253| test loss 0.0704294167 
------------------------------------------------------------------------------------------
| end of epoch   3 | time:  0.54s  | train loss 0.0357993318| test loss 0.0309067947 
------------------------------------------------------------------------------------------
| end of epoch   4 | time:  0.54s  | train loss 0.0201718994| test loss 0.0205095397 
---------------------------------------------------------------------------------------

KeyboardInterrupt: 